In [1]:
import json, os
# %run 'proxy.py' 
from epipolicy.core.epidemic import construct_epidemic
from epipolicy.obj.act import construct_act 

warmup_scenario = r"""
{"status":"start","id":"1","name":"Warmup","session":{"features":{"start_date":"2021-01-01","end_date":"2021-12-31","data_source":"GPWv4","region":"UP","level":1,"year":2020},"interventions":[{"id":"1","name":"Vaccination","notes":"","control_params":[{"id":"1","name":"degree","description":"describe parameter 1 here...","default_value":"0"},{"id":"2","name":"max_capacity","description":"describe parameter 2 here...","default_value":"20000"},{"id":"3","name":"price_per_dose","description":"describe parameter 3 here...","default_value":"20"}],"effect":"def effect(cp, locales):\n    sim.move({'compartment':'S', 'locale':locales}, {'compartment':'V1', 'locale':locales}, cp['degree']*cp['max_capacity'])\n","cost":"def cost(cp, locales):\n    doses = cp['degree']*cp['max_capacity']\n    sim.add({\"intervention\":\"Vaccination\", 'locale':locales}, doses*cp['price_per_dose'])\n"},{"id":"2","name":"Mask","notes":"","control_params":[{"id":"1","name":"compliance","description":"describe parameter 1 here...","default_value":"0"},{"id":"2","name":"cost_per_day","description":"describe parameter 2 here...","default_value":"0.05"}],"effect":"def effect(cp, locales):\n    sim.apply({'parameter':'beta', 'locale':locales}, 1-cp['compliance']*0.7)\n","cost":"def cost(cp, locales):\n    complianceCount = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['compliance']\n    sim.add({\"intervention\":\"Mask\"}, complianceCount*cp['cost_per_day'])\n"}],"costs":[{"id":"1","name":"Infectious_cost","notes":"Infectious people burden the hospital system","intervention":null,"control_params":[{"id":"1","name":"cost_per_day","description":"describe parameter 1 here...","default_value":"100"}],"func":"def cost(cp):\n    infCount = sim.select({'compartment':'{\"tag\":\"infectious\"}'})['Value'].sum()\n    sim.add({\"intervention\":\"Infectious_cost\"}, infCount*cp['cost_per_day'])"}],"locales":[{"population":1012394,"area":99324,"name":"UnitedProvinces","id":"UP","parent_id":""},{"population":202478.8,"area":44619,"name":"UnitedProvinces.Pastures","id":"UP.Pastures","parent_id":"UP"},{"population":313842.14,"area":24504,"name":"UnitedProvinces.Hills","id":"UP.Hills","parent_id":"UP"},{"population":496073.06,"area":30201,"name":"UnitedProvinces.Beaches","id":"UP.Beaches","parent_id":"UP"}],"model":{"name":"Two-dose vaccine SIR","compartments":[{"id":1,"name":"S","desc":"Susceptible","equation":"-(beta * I * S / N) + nu * R - (v1 * S)","tags":["susceptible"]},{"id":2,"name":"I","desc":"Infected","equation":"(beta * I * S / N) + (p1 * beta * I * V1 / N)\n - (gamma * I)","tags":["infectious"]},{"id":3,"name":"R","desc":"Recovered","equation":"gamma * I - (nu * R)","tags":[]},{"id":4,"name":"V1","desc":"One-dose Vaccinated","equation":"v1 * S - v2 * V1 - (p1 * beta * I * V1 / N)","tags":["susceptible"]},{"id":5,"name":"V2","desc":"Two-dose Vaccinated","equation":"v2 * V1","tags":["vaccinated"]}],"parameters":[{"id":1,"name":"beta","desc":"Transmission rate","default_value":"0.3","tags":["transmission"]},{"id":2,"name":"gamma","desc":"Recovery rate","default_value":"0.125","tags":[]},{"id":3,"name":"nu","desc":"Immunity-loss rate","default_value":"0.03","tags":[]},{"id":4,"name":"v1","desc":"One-dose vaccination rate","default_value":"0","tags":[]},{"id":5,"name":"v2","desc":"Two-dose vaccination rate","default_value":"0.0476","tags":[]},{"id":6,"name":"p1","desc":"Reduction from transmission after first dose","default_value":"0.5","tags":[]}]},"initial_info":{"name":"Warmup3","notes":"","initializers":[{"id":1,"locale_regex":"UnitedProvinces.*","group":"*","compartment":"I","value":"100"},{"id":2,"locale_regex":"UnitedProvinces.*","group":"*","compartment":"V1","value":0},{"id":3,"locale_regex":"UnitedProvinces.*","group":"*","compartment":"S","value":"2224426"}]},"groups":[{"name":"Seniors","description":"","locales":[{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.063},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.063},{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.063}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[50,84]}},{"name":"Other","description":"","locales":[{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.936},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.936},{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.936}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[0,49]}}],"group_specifications":[],"groups_locales_parameters":[{"id":0,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.3"},{"id":1,"param":"beta","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.3"},{"id":2,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.3"},{"id":3,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.125"},{"id":4,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.125"},{"id":5,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.125"},{"id":6,"param":"nu","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.03"},{"id":7,"param":"nu","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.03"},{"id":8,"param":"nu","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.03"},{"id":9,"param":"v1","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":10,"param":"v1","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":11,"param":"v1","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":12,"param":"v2","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.0476"},{"id":13,"param":"v2","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.0476"},{"id":14,"param":"v2","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.0476"},{"id":15,"param":"p1","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.5"},{"id":16,"param":"p1","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.5"},{"id":17,"param":"p1","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.5"},{"id":18,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Other","value":"0.3"},{"id":19,"param":"beta","locale":"UnitedProvinces.Hills","group":"Other","value":"0.3"},{"id":20,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Other","value":"0.3"},{"id":21,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Other","value":"0.125"},{"id":22,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Other","value":"0.125"},{"id":23,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Other","value":"0.125"},{"id":24,"param":"nu","locale":"UnitedProvinces.Pastures","group":"Other","value":"0.03"},{"id":25,"param":"nu","locale":"UnitedProvinces.Hills","group":"Other","value":"0.03"},{"id":26,"param":"nu","locale":"UnitedProvinces.Beaches","group":"Other","value":"0.03"},{"id":27,"param":"v1","locale":"UnitedProvinces.Pastures","group":"Other","value":"0"},{"id":28,"param":"v1","locale":"UnitedProvinces.Hills","group":"Other","value":"0"},{"id":29,"param":"v1","locale":"UnitedProvinces.Beaches","group":"Other","value":"0"},{"id":30,"param":"v2","locale":"UnitedProvinces.Pastures","group":"Other","value":"0.0476"},{"id":31,"param":"v2","locale":"UnitedProvinces.Hills","group":"Other","value":"0.0476"},{"id":32,"param":"v2","locale":"UnitedProvinces.Beaches","group":"Other","value":"0.0476"},{"id":33,"param":"p1","locale":"UnitedProvinces.Pastures","group":"Other","value":"0.5"},{"id":34,"param":"p1","locale":"UnitedProvinces.Hills","group":"Other","value":"0.5"},{"id":35,"param":"p1","locale":"UnitedProvinces.Beaches","group":"Other","value":"0.5"}],"facilities":[{"id":1,"name":"Household","description":"The household facility represents the pairwise connections between household members. Unlike schools and workplaces, everyone must be assigned to a household."},{"id":2,"name":"School","description":"The school facility represents all of the pairwise connections between people in schools, including both students and teachers. The current methods in SynthPops treat student and worker status as mutually exclusive."},{"id":3,"name":"Workplace","description":"The workplace facility represents all of the pairwise connections between people in workplaces, except for teachers working in schools. After some workers are assigned to the school contact layer as teachers, all remaining workers are assigned to workplaces. Workplaces are special in that there is little/no age structure so workers of all ages may be present in every workplace."},{"id":4,"name":"Community","description":"The community facility reflects the nature of contacts in shared public spaces like parks and recreational spaces, shopping centres, community centres, and public transportation. All links between individuals are considered undirected to reflect the ability of either individual in the pair to infect each other."}],"facilities_interactions":[{"locales":"UnitedProvinces.*","facilities":[[[1,0],[0,0]],[[0,0],[0,0]],[[1,0],[0,0]],[[1,0],[0,0]]]}],"facilities_timespent":[{"locales":"UnitedProvinces.*","matrix":[[0.53,0],[0,0],[0.19,0],[0.27,0]]}],"schedules":[{"id":"1","name":"Vaccination","notes":"describe schedule for Vaccination...","detail":[{"id":2,"start_date":"2021-01-01","end_date":"2021-02-28","control_params":[{"name":"degree","value":"0"},{"name":"max_capacity","value":"20000"},{"name":"price_per_dose","value":"20"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"2","name":"Mask","notes":"describe schedule for Mask...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"compliance","value":"1"},{"name":"cost_per_day","value":"0.05"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]}],"references":[],"border":{"data":[{"src_locale":"UnitedProvinces.Pastures","dst_locale":"UnitedProvinces.Beaches","src_locale_id":"UP.Pastures","dst_locale_id":"UP.Beaches","group":"*","value":0.4},{"src_locale":"UnitedProvinces.Hills","dst_locale":"UnitedProvinces.Beaches","src_locale_id":"UP.Hills","dst_locale_id":"UP.Beaches","group":"*","value":0.6},{"src_locale":"UnitedProvinces.Beaches","dst_locale":"UnitedProvinces.Pastures","src_locale_id":"UP.Beaches","dst_locale_id":"UP.Pastures","group":"*","value":0.3},{"src_locale":"UnitedProvinces.Beaches","dst_locale":"UnitedProvinces.Hills","src_locale_id":"UP.Beaches","dst_locale_id":"UP.Hills","group":"*","value":0.7}],"specifications":[{"id":1,"src_locale":"UnitedProvinces.*","dst_locale":"UnitedProvinces.*","group":"*","value":0,"impedance":70,"mobility_source":"GPWv4"}]},"airport":{"data":[],"specifications":[]},"facility":{"data":[],"specifications":[]}}}
"""

In [2]:
json_input = json.loads(warmup_scenario)
epi = construct_epidemic(json_input)

<string>:3: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
<string>:3: NumbaTypeSafetyWarning: unsafe cast from float64 to float32. Precision may be lost.
<string>:3: NumbaTypeSafetyWarning: unsafe cast from int64 to float32. Precision may be lost.


In [3]:
state = epi.reset()
state.obs.current_comp.shape

(5, 3, 2)

5: number of compartments 

3: locations 

2: number of (sub-population) groups (senior, adults, children) 

In [4]:
import numpy as np
act = construct_act(0, [0.2,10,10])
state = epi.step([act]) 
state

(<numba.experimental.jitclass.boxing.State at 0x1576b9ee0>,
 -4566.504778395923,
 False)

State, reward, done 

In [5]:
cc = state.obs.current_comp
print(cc.shape)

AttributeError: 'tuple' object has no attribute 'obs'

In [6]:
import argparse
import os
import random
import time
from distutils.util import strtobool

import gym
import numpy as np
import pybullet_envs  # noqa
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.normal import Normal
from torch.utils.tensorboard import SummaryWriter

# Json
SIR_A = r'''{"features":{"data_source":"GPWv4","level":0,"year":2020,"region":"UP","region_name":"UnitedProvinces","start_date":"2021-01-01","end_date":"2021-12-31"},"interventions":[{"name":"Vaccination","notes":"Each day, vaccination directly moves a fixed number of people from susceptible compartment to vaccinated compartment. The number of people vaccinated is determined by the product of 'degree' and 'max_capacity'. Hence, when the 'degree' is 100%, 'max_capacity' people are vaccinated.","control_params":[{"id":"1","name":"degree","description":"0% means no vaccination at all while 100% achieves the max_capacity per day","default_value":"1"},{"id":"2","name":"max_capacity","description":"Maximum number of people vaccinated per day if 100% degree is reached","default_value":"9200"},{"id":"3","name":"price_per_dose","description":"Price per dose (or 2-dose if the vaccine requires 2 doses)","default_value":"40"}],"effect":"def effect(cp, locales):\n\tsim.move({'compartment':'S', 'locale':locales, 'group':'*'}, {'compartment':'R','locale':locales, 'group':'*'}, cp['degree']*cp['max_capacity'])","cost":"def cost(cp, locales):\n\tdoses = cp['degree']*cp['max_capacity']\n\tsim.add({'locale':locales}, doses*cp['price_per_dose'])","id":"1"},{"name":"Masks","notes":"Mask reduces the transmission rate in all facilities except Household (since people do not where mask in household). Mask is modeled by two control parameters 'compliance' and 'max_transmission_reduction'. At 100% 'compliance', there will be a 'max_transmission_reduction' to the transmission rate in all facilities except Household.","control_params":[{"id":"1","name":"compliance","description":"Percentage of people wearing mask","default_value":"0.3"},{"id":"2","name":"cost_per_day","description":"Cost per mask","default_value":"0.05"},{"id":"3","name":"max_transmission_reduction","description":"Maximum percentage of reduction for transmission rate when there is 100% compliance","default_value":"0.5"}],"effect":"def effect(cp, locales):\n\tsim.apply({'parameter':'beta', 'facility':'*', 'locale':locales}, 1-cp['compliance']*cp['max_transmission_reduction'])","cost":"def cost(cp, locales):\n\tcompliance_count = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['compliance']\n\tsim.add({'locale':locales}, compliance_count*cp['cost_per_day'])","id":"2"}],"costs":[{"name":"Infection_Cost","notes":"At each day, there will be a cost incurred for each infectious person ","control_params":[{"id":"1","name":"cost_per_day","description":"Cost of infection per person per day","default_value":"173"}],"func":"def cost(cp):\n\tinfectious_count = sim.select({'compartment':'{\"tag\":\"infectious\"}'})['Value'].sum()\n\tsim.add({}, infectious_count*cp['cost_per_day'])","id":"2"}],"locales":[{"population":2224526,"area":99324,"name":"UnitedProvinces","id":"UP","parent_id":""}],"model":{"compartments":[{"id":1,"name":"S","desc":"Susceptible","equation":"-(beta * I * S / N)","tags":["susceptible"]},{"id":2,"name":"I","desc":"Infected","equation":"(beta * I * S / N) - (gamma * I)","tags":["infectious"]},{"id":3,"name":"R","desc":"Recovered","equation":"gamma * I","tags":[]}],"name":"SIR","parameters":[{"id":1,"name":"beta","desc":"Transmission rate","default_value":"0.2","tags":["transmission"]},{"id":2,"name":"gamma","desc":"Recovery rate","default_value":"0.125","tags":[]}]},"initial_info":{"name":"TestSIR3","notes":"","initializers":[{"id":1,"locale_regex":"UnitedProvinces.*","group":"*","compartment":"I","value":"100"}]},"groups":[],"group_specifications":[],"groups_locales_parameters":[],"facilities":[],"facilities_interactions":[],"facilities_timespent":[],"interaction_specifications":[],"schedules":[{"id":"1","name":"Vaccination","notes":"describe schedule for Vaccination...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-06-30","control_params":[{"name":"degree","value":"0"},{"name":"max_capacity","value":"9200"},{"name":"price_per_dose","value":"40"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"2","name":"Masks","notes":"describe schedule for Masks...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-06-30","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null},{"id":2,"start_date":"2021-07-01","end_date":"2021-12-31","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]}],"references":[],"mobility":[],"mobility_specifications":[],"border":{"data":[],"specifications":[{"id":1,"src_locale":"UnitedProvinces","dst_locale":"UnitedProvinces","group":"*","value":0,"impedance":70,"mobility_source":"GPWv4"}]},"airport":{"data":[],"specifications":[]},"facility":{"data":[],"specifications":[]},"optimize":{"configs":{"max_cost":1000000000000},"interventions":[{"name":"Vaccination","control_params":[{"name":"degree","min_value":0,"max_value":1},{"name":"max_capacity","min_value":9200,"max_value":9200},{"name":"price_per_dose","min_value":40,"max_value":40}]},{"name":"Masks","control_params":[{"name":"compliance","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":0.05,"max_value":0.05},{"name":"max_transmission_reduction","min_value":0.5,"max_value":0.5}]}]}}'''

SIR_B = r'''{"features":{"data_source":"GPWv4","level":0,"year":2020,"region":"UP","region_name":"UnitedProvinces","start_date":"2021-01-01","end_date":"2021-12-31"},"interventions":[{"name":"Vaccination","notes":"Each day, vaccination directly moves a fixed number of people from susceptible compartment to vaccinated compartment. The number of people vaccinated is determined by the product of 'degree' and 'max_capacity'. Hence, when the 'degree' is 100%, 'max_capacity' people are vaccinated.","control_params":[{"id":"1","name":"degree","description":"0% means no vaccination at all while 100% achieves the max_capacity per day","default_value":"1"},{"id":"2","name":"max_capacity","description":"Maximum number of people vaccinated per day if 100% degree is reached","default_value":"9200"},{"id":"3","name":"price_per_dose","description":"Price per dose (or 2-dose if the vaccine requires 2 doses)","default_value":"40"}],"effect":"def effect(cp, locales):\n\tsim.move({'compartment':'S', 'locale':locales, 'group':'*'}, {'compartment':'R','locale':locales, 'group':'*'}, cp['degree']*cp['max_capacity'])","cost":"def cost(cp, locales):\n\tdoses = cp['degree']*cp['max_capacity']\n\tsim.add({'locale':locales}, doses*cp['price_per_dose'])","id":"1"},{"name":"Masks","notes":"Mask reduces the transmission rate in all facilities except Household (since people do not where mask in household). Mask is modeled by two control parameters 'compliance' and 'max_transmission_reduction'. At 100% 'compliance', there will be a 'max_transmission_reduction' to the transmission rate in all facilities except Household.","control_params":[{"id":"1","name":"compliance","description":"Percentage of people wearing mask","default_value":"0.3"},{"id":"2","name":"cost_per_day","description":"Cost per mask","default_value":"0.05"},{"id":"3","name":"max_transmission_reduction","description":"Maximum percentage of reduction for transmission rate when there is 100% compliance","default_value":"0.5"}],"effect":"def effect(cp, locales):\n\tsim.apply({'parameter':'beta', 'facility':'*', 'locale':locales}, 1-cp['compliance']*cp['max_transmission_reduction'])","cost":"def cost(cp, locales):\n\tcompliance_count = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['compliance']\n\tsim.add({'locale':locales}, compliance_count*cp['cost_per_day'])","id":"2"},{"id":"3","name":"School closure","notes":"Close school","control_params":[{"id":"1","name":"percentage","description":"The percentage of students being remote","default_value":"0"},{"id":"2","name":"cost_per_day","description":"Economic cost of affecting one student per day","default_value":"1.8"}],"effect":"def effect(cp, locales):\n    sim.apply({\"facility\": \"School\", \"locale\":locales, \"group-from\":\"*\", \"group-to\":\"*\"}, 1-cp['percentage'])","cost":"def cost(cp, locales):\n    affectedCount = sim.select({'compartment':'*', 'locale':locales, 'group':\"Children\"})['Value'].sum() * cp['percentage']\n    sim.add({'locale':locales}, affectedCount*cp['cost_per_day'])"},{"id":"4","name":"Workplace closure","notes":"Close workplace","control_params":[{"id":"1","name":"percentage","description":"describe parameter 1 here...","default_value":"0"},{"id":"2","name":"cost_per_day","description":"describe parameter 2 here...","default_value":"1.8"}],"effect":"def effect(cp, locales):\n    sim.apply({\"facility\": \"Workplace\", \"locale\":locales, \"group-from\":\"*\", \"group-to\":\"*\"}, 1-cp['percentage'])\n","cost":"def cost(cp, locales):\n    affectedCount = sim.select({'compartment':'*', 'locale':locales, 'group':\"Adults\"})['Value'].sum() * cp['percentage']\n    sim.add({'locale':locales}, affectedCount*cp['cost_per_day'])\n"}],"costs":[{"name":"Infection_Cost","notes":"At each day, there will be a cost incurred for each infectious person ","control_params":[{"id":"1","name":"cost_per_day","description":"Cost of infection per person per day","default_value":"173"}],"func":"def cost(cp):\n\tinfectious_count = sim.select({'compartment':'{\"tag\":\"infectious\"}'})['Value'].sum()\n\tsim.add({}, infectious_count*cp['cost_per_day'])","id":"2"}],"locales":[{"population":2224526,"area":99324,"name":"UnitedProvinces","id":"UP","parent_id":""}],"model":{"compartments":[{"id":1,"name":"S","desc":"Susceptible","equation":"-(beta * I * S / N)","tags":["susceptible"]},{"id":2,"name":"I","desc":"Infected","equation":"(beta * I * S / N) - (gamma * I)","tags":["infectious"]},{"id":3,"name":"R","desc":"Recovered","equation":"gamma * I","tags":[]}],"name":"SIR","parameters":[{"id":1,"name":"beta","desc":"Transmission rate","default_value":"0.2","tags":["transmission"]},{"id":2,"name":"gamma","desc":"Recovery rate","default_value":"0.125","tags":[]}]},"initial_info":{"name":"TestSIR7","notes":"","initializers":[{"id":1,"locale_regex":"UnitedProvinces.*","group":"*","compartment":"I","value":"100"}]},"groups":[{"name":"Children","description":"","locales":[{"name":"UnitedProvinces","id":"UP","population":0.251}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[0,19]}},{"name":"Seniors","description":"","locales":[{"name":"UnitedProvinces","id":"UP","population":0.013}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[60,84]}},{"name":"Adults","description":"","locales":[{"name":"UnitedProvinces","id":"UP","population":0.735}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[20,59]}}],"group_specifications":[],"groups_locales_parameters":[{"id":0,"param":"beta","locale":"UnitedProvinces","group":"Children","value":"0.2"},{"id":1,"param":"gamma","locale":"UnitedProvinces","group":"Children","value":"0.125"},{"id":2,"param":"beta","locale":"UnitedProvinces","group":"Seniors","value":"0.2"},{"id":3,"param":"gamma","locale":"UnitedProvinces","group":"Seniors","value":"0.125"},{"id":4,"param":"beta","locale":"UnitedProvinces","group":"Adults","value":"0.2"},{"id":5,"param":"gamma","locale":"UnitedProvinces","group":"Adults","value":"0.125"}],"facilities":[{"id":1,"name":"Household","description":"The household facility represents the pairwise connections between household members. Unlike schools and workplaces, everyone must be assigned to a household."},{"id":2,"name":"School","description":"The school facility represents all of the pairwise connections between people in schools, including both students and teachers. The current methods in SynthPops treat student and worker status as mutually exclusive."},{"id":3,"name":"Workplace","description":"The workplace facility represents all of the pairwise connections between people in workplaces, except for teachers working in schools. After some workers are assigned to the school contact layer as teachers, all remaining workers are assigned to workplaces. Workplaces are special in that there is little/no age structure so workers of all ages may be present in every workplace."},{"id":4,"name":"Community","description":"The community facility reflects the nature of contacts in shared public spaces like parks and recreational spaces, shopping centres, community centres, and public transportation. All links between individuals are considered undirected to reflect the ability of either individual in the pair to infect each other."}],"facilities_interactions":[{"locales":"UnitedProvinces.*","facilities":[[[0.37,0.1,0.53],[0.27,0.37,0.36],[0.32,0.07,0.6]],[[0.92,0,0.08],[0.75,0,0.25],[0.67,0,0.33]],[[0,0.11,0.89],[0.04,0.02,0.94],[0.03,0.05,0.92]],[[0.54,0.06,0.4],[0.1,0.34,0.57],[0.14,0.13,0.73]]]}],"facilities_timespent":[{"locales":"UnitedProvinces.*","matrix":[[0.4,0.54,0.4],[0.31,0.01,0.08],[0.08,0.18,0.32],[0.2,0.27,0.2]]}],"interaction_specifications":[],"schedules":[{"id":"1","name":"Vaccination","notes":"describe schedule for Vaccination...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-06-30","control_params":[{"name":"degree","value":"0"},{"name":"max_capacity","value":"9200"},{"name":"price_per_dose","value":"40"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"2","name":"Masks","notes":"describe schedule for Masks...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-06-30","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null},{"id":2,"start_date":"2021-07-01","end_date":"2021-12-31","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"3","name":"School closure","notes":"describe schedule for School closure...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"percentage","value":"1"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"4","name":"Workplace closure","notes":"describe schedule for Workplace closure...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"percentage","value":"1"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]}],"references":[],"mobility":[],"mobility_specifications":[],"border":{"data":[],"specifications":[{"id":1,"src_locale":"UnitedProvinces","dst_locale":"UnitedProvinces","group":"*","value":0,"impedance":70,"mobility_source":"GPWv4"}]},"airport":{"data":[],"specifications":[]},"facility":{"data":[],"specifications":[]},"optimize":{"configs":{"max_cost":1000000000000},"interventions":[{"name":"Vaccination","control_params":[{"name":"degree","min_value":0,"max_value":1},{"name":"max_capacity","min_value":9200,"max_value":9200},{"name":"price_per_dose","min_value":40,"max_value":40}]},{"name":"Masks","control_params":[{"name":"compliance","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":0.05,"max_value":0.05},{"name":"max_transmission_reduction","min_value":0.5,"max_value":0.5}]},{"name":"School closure","control_params":[{"name":"percentage","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]},{"name":"Workplace closure","control_params":[{"name":"percentage","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]}]}}'''

# Debug  
SIRV_A = r'''{"features": {"data_source": "GPWv4", "level": 0, "year": 2020, "region": "UP", "region_name": "UnitedProvinces", "start_date": "2021-01-01", "end_date": "2021-12-31"}, "interventions": [{"name": "Vaccination", "notes": "Each day, vaccination directly moves a fixed number of people from susceptible compartment to vaccinated compartment. The number of people vaccinated is determined by the product of 'degree' and 'max_capacity'. Hence, when the 'degree' is 100%, 'max_capacity' people are vaccinated.", "control_params": [{"id": "1", "name": "degree", "description": "0% means no vaccination at all while 100% achieves the max_capacity per day", "default_value": "1"}, {"id": "2", "name": "max_capacity", "description": "Maximum number of people vaccinated per day if 100% degree is reached", "default_value": "9200"}, {"id": "3", "name": "price_per_dose", "description": "Price per dose (or 2-dose if the vaccine requires 2 doses)", "default_value": "40"}], "effect": "def effect(cp, locales):\n\tsim.move({'compartment':'S', 'locale':locales, 'group':'*'}, {'compartment':'V1','locale':locales, 'group':'*'}, cp['degree']*cp['max_capacity'])", "cost": "def cost(cp, locales):\n\tdoses = cp['degree']*cp['max_capacity']\n\tsim.add({'locale':locales}, doses*cp['price_per_dose'])", "id": "1"}, {"name": "Masks", "notes": "Mask reduces the transmission rate in all facilities except Household (since people do not where mask in household). Mask is modeled by two control parameters 'compliance' and 'max_transmission_reduction'. At 100% 'compliance', there will be a 'max_transmission_reduction' to the transmission rate in all facilities except Household.", "control_params": [{"id": "1", "name": "compliance", "description": "Percentage of people wearing mask", "default_value": "0.3"}, {"id": "2", "name": "cost_per_day", "description": "Cost per mask", "default_value": "0.05"}, {"id": "3", "name": "max_transmission_reduction", "description": "Maximum percentage of reduction for transmission rate when there is 100% compliance", "default_value": "0.5"}], "effect": "def effect(cp, locales):\n\tsim.apply({'parameter':'beta', 'facility':'*', 'locale':locales}, 1-cp['compliance']*cp['max_transmission_reduction'])", "cost": "def cost(cp, locales):\n\tcompliance_count = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['compliance']\n\tsim.add({'locale':locales}, compliance_count*cp['cost_per_day'])", "id": "2"}], "costs": [{"name": "Infection_Cost", "notes": "At each day, there will be a cost incurred for each infectious person ", "control_params": [{"id": "1", "name": "cost_per_day", "description": "Cost of infection per person per day", "default_value": "173"}], "func": "def cost(cp):\n\tinfectious_count = sim.select({'compartment':'{\"tag\":\"infectious\"}'})['Value'].sum()\n\tsim.add({}, infectious_count*cp['cost_per_day'])", "id": "2"}], "locales": [{"population": 2224526, "area": 99324, "name": "UnitedProvinces", "id": "UP", "parent_id": ""}], "model": {"name": "Two-dose vaccine SIR", "compartments": [{"id": 1, "name": "S", "desc": "Susceptible", "equation": "-(beta * I * S / N) + nu * R - (v1 * S)", "tags": ["susceptible"]}, {"id": 2, "name": "I", "desc": "Infected", "equation": "(beta * I * S / N) + (p1 * beta * I * V1 / N)\n - (gamma * I)", "tags": ["infectious"]}, {"id": 3, "name": "R", "desc": "Recovered", "equation": "gamma * I - (nu * R)", "tags": []}, {"id": 4, "name": "V1", "desc": "One-dose Vaccinated", "equation": "v1 * S - v2 * V1 - (p1 * beta * I * V1 / N)", "tags": ["susceptible"]}, {"id": 5, "name": "V2", "desc": "Two-dose Vaccinated", "equation": "v2 * V1", "tags": ["vaccinated"]}], "parameters": [{"id": 1, "name": "beta", "desc": "Transmission rate", "default_value": "0.2", "tags": ["transmission"]}, {"id": 2, "name": "gamma", "desc": "Recovery rate", "default_value": "0.05", "tags": []}, {"id": 3, "name": "nu", "desc": "Immunity-loss rate", "default_value": "0.01", "tags": []}, {"id": 4, "name": "v1", "desc": "One-dose vaccination rate", "default_value": "0", "tags": []}, {"id": 5, "name": "v2", "desc": "Two-dose vaccination rate", "default_value": "0.0476", "tags": []}, {"id": 6, "name": "p1", "desc": "Reduction from transmission after first dose", "default_value": "0.5", "tags": []}]}, "initial_info": {"name": "Test5", "notes": "", "initializers": [{"id": 1, "locale_regex": "UnitedProvinces.*", "group": "*", "compartment": "I", "value": "100"}]}, "groups": [], "group_specifications": [], "groups_locales_parameters": [], "facilities": [], "facilities_interactions": [], "facilities_timespent": [], "interaction_specifications": [], "schedules": [{"id": "1", "name": "Vaccination", "notes": "describe schedule for Vaccination...", "detail": [{"id": 1, "start_date": "2021-01-01", "end_date": "2021-06-30", "control_params": [{"name": "degree", "value": "1"}, {"name": "max_capacity", "value": "9200"}, {"name": "price_per_dose", "value": "40"}], "locales": "UnitedProvinces.*", "has_trigger": false, "condition": "def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t", "repeat_config": {"repeat_type": "none", "end_type": "never", "end_date": "2021-12-31", "end_times": 0}, "repetition_of": null}]}, {"id": "2", "name": "Masks", "notes": "describe schedule for Masks...", "detail": [{"id": 1, "start_date": "2021-01-01", "end_date": "2021-06-30", "control_params": [{"name": "compliance", "value": "1"}, {"name": "cost_per_day", "value": "0.05"}, {"name": "max_transmission_reduction", "value": "0.5"}], "locales": "UnitedProvinces.*", "has_trigger": false, "condition": "def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t", "repeat_config": {"repeat_type": "none", "end_type": "never", "end_date": "2021-12-31", "end_times": 0}, "repetition_of": null}, {"id": 2, "start_date": "2021-07-01", "end_date": "2021-12-31", "control_params": [{"name": "compliance", "value": "0.3"}, {"name": "cost_per_day", "value": "0.05"}, {"name": "max_transmission_reduction", "value": "0.5"}], "locales": "UnitedProvinces.*", "has_trigger": false, "condition": "def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t", "repeat_config": {"repeat_type": "none", "end_type": "never", "end_date": "2021-12-31", "end_times": 0}, "repetition_of": null}]}], "references": [], "mobility": [], "mobility_specifications": [], "border": {"data": [], "specifications": [{"id": 1, "src_locale": "UnitedProvinces", "dst_locale": "UnitedProvinces", "group": "*", "value": 0, "impedance": 70, "mobility_source": "GPWv4"}]}, "airport": {"data": [], "specifications": []}, "facility": {"data": [], "specifications": []}, "optimize": {"configs": {"max_cost": 1000000000000}, "interventions": [{"name": "Vaccination", "control_params": [{"name": "degree", "min_value": 0, "max_value": 1}, {"name": "max_capacity", "min_value": 9200.0, "max_value": 9200.0}, {"name": "price_per_dose", "min_value": 40.0, "max_value": 40.0}]}, {"name": "Masks", "control_params": [{"name": "compliance", "min_value": 0, "max_value": 1}, {"name": "cost_per_day", "min_value": 0.05, "max_value": 0.05}, {"name": "max_transmission_reduction", "min_value": 0.5, "max_value": 0.5}]}]}}'''

SIRV_B = r'''{"features":{"data_source":"GPWv4","level":0,"year":2020,"region":"UP","region_name":"UnitedProvinces","start_date":"2021-01-01","end_date":"2021-12-31"},"interventions":[{"name":"Vaccination","notes":"Each day, vaccination directly moves a fixed number of people from susceptible compartment to vaccinated compartment. The number of people vaccinated is determined by the product of 'degree' and 'max_capacity'. Hence, when the 'degree' is 100%, 'max_capacity' people are vaccinated.","control_params":[{"id":"1","name":"degree","description":"0% means no vaccination at all while 100% achieves the max_capacity per day","default_value":"1"},{"id":"2","name":"max_capacity","description":"Maximum number of people vaccinated per day if 100% degree is reached","default_value":"9200"},{"id":"3","name":"price_per_dose","description":"Price per dose (or 2-dose if the vaccine requires 2 doses)","default_value":"40"}],"effect":"def effect(cp, locales):\n\tsim.move({'compartment':'S', 'locale':locales, 'group':'*'}, {'compartment':'V1','locale':locales, 'group':'*'}, cp['degree']*cp['max_capacity'])","cost":"def cost(cp, locales):\n\tdoses = cp['degree']*cp['max_capacity']\n\tsim.add({'locale':locales}, doses*cp['price_per_dose'])","id":"1"},{"name":"Masks","notes":"Mask reduces the transmission rate in all facilities except Household (since people do not where mask in household). Mask is modeled by two control parameters 'compliance' and 'max_transmission_reduction'. At 100% 'compliance', there will be a 'max_transmission_reduction' to the transmission rate in all facilities except Household.","control_params":[{"id":"1","name":"compliance","description":"Percentage of people wearing mask","default_value":"0.3"},{"id":"2","name":"cost_per_day","description":"Cost per mask","default_value":"0.05"},{"id":"3","name":"max_transmission_reduction","description":"Maximum percentage of reduction for transmission rate when there is 100% compliance","default_value":"0.5"}],"effect":"def effect(cp, locales):\n\tsim.apply({'parameter':'beta', 'facility':'*', 'locale':locales}, 1-cp['compliance']*cp['max_transmission_reduction'])","cost":"def cost(cp, locales):\n\tcompliance_count = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['compliance']\n\tsim.add({'locale':locales}, compliance_count*cp['cost_per_day'])","id":"2"},{"id":"3","name":"School closure","notes":"Close school","control_params":[{"id":"1","name":"percentage","description":"The percentage of students being remote","default_value":"0"},{"id":"2","name":"cost_per_day","description":"Economic cost of affecting one student per day","default_value":"1.8"}],"effect":"def effect(cp, locales):\n    sim.apply({\"facility\": \"School\", \"locale\":locales, \"group-from\":\"*\", \"group-to\":\"*\"}, 1-cp['percentage'])","cost":"def cost(cp, locales):\n    affectedCount = sim.select({'compartment':'*', 'locale':locales, 'group':\"Children\"})['Value'].sum() * cp['percentage']\n    sim.add({'locale':locales}, affectedCount*cp['cost_per_day'])"},{"id":"4","name":"Workplace closure","notes":"Close workplace","control_params":[{"id":"1","name":"percentage","description":"describe parameter 1 here...","default_value":"0"},{"id":"2","name":"cost_per_day","description":"describe parameter 2 here...","default_value":"1.8"}],"effect":"def effect(cp, locales):\n    sim.apply({\"facility\": \"Workplace\", \"locale\":locales, \"group-from\":\"*\", \"group-to\":\"*\"}, 1-cp['percentage'])\n","cost":"def cost(cp, locales):\n    affectedCount = sim.select({'compartment':'*', 'locale':locales, 'group':\"Adults\"})['Value'].sum() * cp['percentage']\n    sim.add({'locale':locales}, affectedCount*cp['cost_per_day'])\n"}],"costs":[{"name":"Infection_Cost","notes":"At each day, there will be a cost incurred for each infectious person ","control_params":[{"id":"1","name":"cost_per_day","description":"Cost of infection per person per day","default_value":"173"}],"func":"def cost(cp):\n\tinfectious_count = sim.select({'compartment':'{\"tag\":\"infectious\"}'})['Value'].sum()\n\tsim.add({}, infectious_count*cp['cost_per_day'])","id":"2"}],"locales":[{"population":2224526,"area":99324,"name":"UnitedProvinces","id":"UP","parent_id":""}],"model":{"name":"Two-dose vaccine SIR","compartments":[{"id":1,"name":"S","desc":"Susceptible","equation":"-(beta * I * S / N) + nu * R - (v1 * S)","tags":["susceptible"]},{"id":2,"name":"I","desc":"Infected","equation":"(beta * I * S / N) + (p1 * beta * I * V1 / N)\n - (gamma * I)","tags":["infectious"]},{"id":3,"name":"R","desc":"Recovered","equation":"gamma * I - (nu * R)","tags":[]},{"id":4,"name":"V1","desc":"One-dose Vaccinated","equation":"v1 * S - v2 * V1 - (p1 * beta * I * V1 / N)","tags":["susceptible"]},{"id":5,"name":"V2","desc":"Two-dose Vaccinated","equation":"v2 * V1","tags":["vaccinated"]}],"parameters":[{"id":1,"name":"beta","desc":"Transmission rate","default_value":"0.2","tags":["transmission"]},{"id":2,"name":"gamma","desc":"Recovery rate","default_value":"0.05","tags":[]},{"id":3,"name":"nu","desc":"Immunity-loss rate","default_value":"0.01","tags":[]},{"id":4,"name":"v1","desc":"One-dose vaccination rate","default_value":"0","tags":[]},{"id":5,"name":"v2","desc":"Two-dose vaccination rate","default_value":"0.0476","tags":[]},{"id":6,"name":"p1","desc":"Reduction from transmission after first dose","default_value":"0.5","tags":[]}]},"initial_info":{"name":"Test11","notes":"","initializers":[{"id":1,"locale_regex":"UnitedProvinces.*","group":"*","compartment":"I","value":"100"}]},"groups":[{"name":"Children","description":"","locales":[{"name":"UnitedProvinces","id":"UP","population":0.251}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[0,19]}},{"name":"Adults","description":"","locales":[{"name":"UnitedProvinces","id":"UP","population":0.735}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[20,59]}},{"name":"Seniors","description":"","locales":[{"name":"UnitedProvinces","id":"UP","population":0.013}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[60,84]}}],"group_specifications":[],"groups_locales_parameters":[{"id":0,"param":"beta","locale":"UnitedProvinces","group":"Children","value":"0.2"},{"id":1,"param":"gamma","locale":"UnitedProvinces","group":"Children","value":"0.05"},{"id":2,"param":"nu","locale":"UnitedProvinces","group":"Children","value":"0.01"},{"id":3,"param":"v1","locale":"UnitedProvinces","group":"Children","value":"0"},{"id":4,"param":"v2","locale":"UnitedProvinces","group":"Children","value":"0.0476"},{"id":5,"param":"p1","locale":"UnitedProvinces","group":"Children","value":"0.5"},{"id":6,"param":"beta","locale":"UnitedProvinces","group":"Adults","value":"0.2"},{"id":7,"param":"gamma","locale":"UnitedProvinces","group":"Adults","value":"0.05"},{"id":8,"param":"nu","locale":"UnitedProvinces","group":"Adults","value":"0.01"},{"id":9,"param":"v1","locale":"UnitedProvinces","group":"Adults","value":"0"},{"id":10,"param":"v2","locale":"UnitedProvinces","group":"Adults","value":"0.0476"},{"id":11,"param":"p1","locale":"UnitedProvinces","group":"Adults","value":"0.5"},{"id":12,"param":"beta","locale":"UnitedProvinces","group":"Seniors","value":"0.2"},{"id":13,"param":"gamma","locale":"UnitedProvinces","group":"Seniors","value":"0.05"},{"id":14,"param":"nu","locale":"UnitedProvinces","group":"Seniors","value":"0.01"},{"id":15,"param":"v1","locale":"UnitedProvinces","group":"Seniors","value":"0"},{"id":16,"param":"v2","locale":"UnitedProvinces","group":"Seniors","value":"0.0476"},{"id":17,"param":"p1","locale":"UnitedProvinces","group":"Seniors","value":"0.5"}],"facilities":[{"id":1,"name":"Household","description":"The household facility represents the pairwise connections between household members. Unlike schools and workplaces, everyone must be assigned to a household."},{"id":2,"name":"School","description":"The school facility represents all of the pairwise connections between people in schools, including both students and teachers. The current methods in SynthPops treat student and worker status as mutually exclusive."},{"id":3,"name":"Workplace","description":"The workplace facility represents all of the pairwise connections between people in workplaces, except for teachers working in schools. After some workers are assigned to the school contact layer as teachers, all remaining workers are assigned to workplaces. Workplaces are special in that there is little/no age structure so workers of all ages may be present in every workplace."},{"id":4,"name":"Community","description":"The community facility reflects the nature of contacts in shared public spaces like parks and recreational spaces, shopping centres, community centres, and public transportation. All links between individuals are considered undirected to reflect the ability of either individual in the pair to infect each other."}],"facilities_interactions":[{"locales":"UnitedProvinces.*","facilities":[[[0.37,0.53,0.1],[0.32,0.6,0.07],[0.27,0.36,0.37]],[[0.92,0.08,0],[0.67,0.33,0],[0.75,0.25,0]],[[0,0.89,0.11],[0.03,0.92,0.05],[0.04,0.94,0.02]],[[0.54,0.4,0.06],[0.14,0.73,0.13],[0.1,0.57,0.34]]]}],"facilities_timespent":[{"locales":"UnitedProvinces.*","matrix":[[0.4,0.4,0.54],[0.31,0.08,0.01],[0.08,0.32,0.18],[0.2,0.2,0.27]]}],"interaction_specifications":[],"schedules":[{"id":"1","name":"Vaccination","notes":"describe schedule for Vaccination...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-06-30","control_params":[{"name":"degree","value":"1"},{"name":"max_capacity","value":"9200"},{"name":"price_per_dose","value":"40"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"2","name":"Masks","notes":"describe schedule for Masks...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-06-30","control_params":[{"name":"compliance","value":"1"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null},{"id":2,"start_date":"2021-07-01","end_date":"2021-12-31","control_params":[{"name":"compliance","value":"0.3"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"3","name":"School closure","notes":"describe schedule for School closure...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"percentage","value":"0.2"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"4","name":"Workplace closure","notes":"describe schedule for Workplace closure...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"percentage","value":"0.2"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]}],"references":[],"mobility":[],"mobility_specifications":[],"border":{"data":[],"specifications":[{"id":1,"src_locale":"UnitedProvinces","dst_locale":"UnitedProvinces","group":"*","value":0,"impedance":70,"mobility_source":"GPWv4"}]},"airport":{"data":[],"specifications":[]},"facility":{"data":[],"specifications":[]},"optimize":{"configs":{"max_cost":1000000000000},"interventions":[{"name":"Vaccination","control_params":[{"name":"degree","min_value":0,"max_value":1},{"name":"max_capacity","min_value":9200,"max_value":9200},{"name":"price_per_dose","min_value":40,"max_value":40}]},{"name":"Masks","control_params":[{"name":"compliance","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":0.05,"max_value":0.05},{"name":"max_transmission_reduction","min_value":0.5,"max_value":0.5}]},{"name":"School closure","control_params":[{"name":"percentage","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]},{"name":"Workplace closure","control_params":[{"name":"percentage","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]}]}}'''

# Slow 
COVID_A = r'''{"features":{"data_source":"GPWv4","region":"UP","region_name":"UnitedProvinces","level":1,"year":2020,"start_date":"2021-01-01","end_date":"2021-12-31"},"interventions":[{"name":"Vaccination","notes":"Each day, vaccination directly moves a fixed number of people from susceptible compartment to vaccinated compartment. The number of people vaccinated is determined by the product of 'degree' and 'max_capacity'. Hence, when the 'degree' is 100%, 'max_capacity' people are vaccinated.","control_params":[{"id":"1","name":"degree","description":"0% means no vaccination at all while 100% achieves the max_capacity per day","default_value":"0"},{"id":"2","name":"max_capacity","description":"Maximum number of people vaccinated per day if 100% degree is reached","default_value":"50000"},{"id":"3","name":"price_per_dose","description":"Price per dose (or 2-dose if the vaccine requires 2 doses)","default_value":"40"}],"effect":"def effect(cp, locales):\n\tsim.move({'compartment':'S', 'locale':locales, 'group':'*'}, {'compartment':'V','locale':locales, 'group':'*'}, cp['degree']*cp['max_capacity'])","cost":"def cost(cp, locales):\n\tdoses = cp['degree']*cp['max_capacity']\n\tsim.add({'locale':locales}, doses*cp['price_per_dose'])","id":"5"},{"name":"Masks","notes":"Mask reduces the transmission rate in all facilities except Household (since people do not where mask in household). Mask is modeled by two control parameters 'compliance' and 'max_transmission_reduction'. At 100% 'compliance', there will be a 'max_transmission_reduction' to the transmission rate in all facilities except Household.","control_params":[{"id":"1","name":"compliance","description":"Percentage of people wearing mask","default_value":"0"},{"id":"2","name":"cost_per_day","description":"Cost per mask","default_value":"0.05"},{"id":"3","name":"max_transmission_reduction","description":"Maximum percentage of reduction for transmission rate when there is 100% compliance","default_value":"0.5"}],"effect":"def effect(cp, locales):\n\tsim.apply({'parameter':'beta', 'facility':'~Household', 'locale':locales}, 1-cp['compliance']*cp['max_transmission_reduction'])","cost":"def cost(cp, locales):\n\tcompliance_count = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['compliance']\n\tsim.add({'locale':locales}, compliance_count*cp['cost_per_day'])","id":"3"}],"costs":[{"id":"1","name":"Infectious_cost","notes":"","intervention":null,"control_params":[{"id":"1","name":"cost_per_day","description":"describe parameter 1 here...","default_value":"173"}],"func":"def cost(cp):\n    infectious_count = sim.select({'compartment':'{\"tag\":\"infectious\"}'})['Value'].sum()\n    sim.add({\"intervention\":\"Infectious_cost\"}, infectious_count*cp['cost_per_day'])"},{"id":"2","name":"Hospitalized_cost","notes":"","intervention":null,"control_params":[{"id":"1","name":"cost_per_day","description":"describe parameter 1 here...","default_value":"250"}],"func":"def cost(cp):\n    hospitalized_count = sim.select({'compartment':'{\"tag\":\"hospitalized\"}'})['Value'].sum()\n    sim.add({\"intervention\":\"Hospitalized_cost\"}, hospitalized_count*cp['cost_per_day'])"},{"id":"3","name":"Death_cost","notes":"","intervention":null,"control_params":[{"id":"1","name":"cost_per_death","description":"describe parameter 1 here...","default_value":"100000"}],"func":"def cost(cp):\n    death_increase = sim.select({'compartment':'D', 'value-mode':'change'})['Value'].sum()\n    sim.add({\"intervention\":\"Death_cost\"}, death_increase*cp['cost_per_death'])"}],"locales":[{"population":1012394,"area":99324,"name":"UnitedProvinces","id":"UP","parent_id":""},{"population":496073.06,"area":30201,"name":"UnitedProvinces.Beaches","id":"UP.Beaches","parent_id":"UP"},{"population":313842.14,"area":24504,"name":"UnitedProvinces.Hills","id":"UP.Hills","parent_id":"UP"},{"population":202478.8,"area":44619,"name":"UnitedProvinces.Pastures","id":"UP.Pastures","parent_id":"UP"}],"model":{"name":"COVID19-Model","compartments":[{"id":1,"name":"S","desc":"Susceptible","equation":"gamma*R - vs*S + iqs*Qs - is*S*(Iasym + Ipre + Imild + Qasym + Qpre + Hmild + Hsev + Hcri) / N - beta * (Iasym + Ipre + Imild) * S / N - 0.2 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * S / N","tags":["susceptible"]},{"id":2,"name":"E","desc":"Exposed","equation":"-(ie*E) -(1-psym)*sigma*E -(psym*sigma*E) + beta * (Iasym + Ipre + Imild) * S / N + 0.2 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * S / N","tags":["exposed"]},{"id":3,"name":"Ipre","desc":"Presymptomatic","equation":"psym*sigma*E-ipre*Ipre-ri*Ipre","tags":["infectious"]},{"id":4,"name":"Iasym","desc":"Asymptomatic","equation":"(1-psym)*sigma*E -iasym*Iasym -ra*Iasym","tags":["infectious"]},{"id":5,"name":"Imild","desc":"Mild","equation":"ri*Ipre -imild*Imild -psev*rm*Imild -(1-psev)*rm*Imild","tags":["infectious"]},{"id":6,"name":"Hsev","desc":"Severe with Hospitalization","equation":"psev*rm*Imild + 0.5*psev*rm*Hmild -pcri*rs*Hsev -(1-pcri)*rs*Hsev","tags":["infectious","hospitalized"]},{"id":7,"name":"Hcri","desc":"Critical with Hospitalization","equation":"pcri*rs*Hsev -pdeath*rc*Hcri -(1-pdeath)*rc*Hcri","tags":["infectious","hospitalized"]},{"id":8,"name":"D","desc":"Death","equation":"pdeath*rc*Hcri","tags":["death"]},{"id":9,"name":"R","desc":"Recovered","equation":"-(gamma*R) + ra*Iasym + (1-psev)*rm*Imild + ra*Qasym + (1-0.5*psev)*rm*Hmild + (1-pcri)*rs*Hsev + (1-pdeath)*rc*Hcri","tags":["recovered"]},{"id":10,"name":"Qs","desc":"Quarantined susceptible","equation":"-vqs*Qs + is*S*(Iasym + Ipre + Imild + Qasym + Qpre + Hmild + Hsev + Hcri) / N -iqs*Qs - 0.12 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * Qs / N - 0.6 * beta * (Iasym + Ipre + Imild) * Qs / N","tags":["susceptible","isolated"]},{"id":11,"name":"Qe","desc":"Quarantined exposed","equation":"ie*E -psym*sigma*Qe -(1-psym)*sigma*Qe + 0.12 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * Qs / N + 0.6 * beta * (Iasym + Ipre + Imild) * Qs / N","tags":["exposed","isolated"]},{"id":12,"name":"Qpre","desc":"Isolated presymptomatic","equation":"psym*sigma*Qe + ipre*Ipre -ri*Qpre","tags":["infectious","isolated"]},{"id":13,"name":"Qasym","desc":"Isolated asymptomatic","equation":"(1-psym)*sigma*Qe + iasym*Iasym -ra*Qasym","tags":["infectious","isolated"]},{"id":14,"name":"Hmild","desc":"Hospitalized mild","equation":"ri*Qpre + imild*Imild -0.5*psev*rm*Hmild -(1-0.5*psev)*rm*Hmild","tags":["infectious","hospitalized"]},{"id":15,"name":"V","desc":"Vaccinated","equation":"vs*S+vqs*Qs","tags":["vaccinated"]}],"parameters":[{"id":1,"name":"beta","desc":"Transmission rate","default_value":"0.27","tags":["transmission"]},{"id":2,"name":"sigma","desc":"Incubation rate","default_value":"0.21739","tags":[]},{"id":3,"name":"psym","desc":"Probability of having symptoms","default_value":"0.63658","tags":[]},{"id":4,"name":"psev","desc":"Probability of developing serious complications","default_value":"0.04865","tags":[]},{"id":5,"name":"pcri","desc":"Probability of developing critical complications","default_value":"0.08557","tags":[]},{"id":6,"name":"pdeath","desc":"Probability of death","default_value":"0.75472","tags":[]},{"id":7,"name":"ra","desc":"Asymptomatic recovery rate","default_value":"0.05","tags":[]},{"id":8,"name":"ri","desc":"Viral shedding rate","default_value":"0.75188","tags":[]},{"id":9,"name":"rm","desc":"Mild recovery rate","default_value":"0.125","tags":[]},{"id":10,"name":"rs","desc":"Severe recovery rate","default_value":"0.07143","tags":[]},{"id":11,"name":"rc","desc":"Critical recovery rate","default_value":"0.07143","tags":[]},{"id":12,"name":"is","desc":"Susceptible quarantine rate","default_value":"0.02643","tags":["quarantine"]},{"id":13,"name":"iqs","desc":"Susceptible end-of-quarantine rate","default_value":"0.07143","tags":[]},{"id":14,"name":"ie","desc":"Exposed quarantine rate","default_value":"0.09301","tags":["quarantine"]},{"id":15,"name":"ipre","desc":"Presymptomatic quarantine rate","default_value":"0.16179","tags":["quarantine"]},{"id":16,"name":"iasym","desc":"Asymptomatic quarantine rate","default_value":"0.16179","tags":["quarantine"]},{"id":17,"name":"imild","desc":"Mild hospitalized rate","default_value":"0.19231","tags":[]},{"id":19,"name":"vs","desc":"Daily percentage of susceptible being vaccinated","default_value":"0","tags":[]},{"id":20,"name":"vqs","desc":"Daily percentage of quarantine-susceptible being vaccinated","default_value":"0","tags":[]},{"id":21,"name":"gamma","desc":"Immunity-loss rate","default_value":"0.0333","tags":[]}]},"initial_info":{"name":"UpMV24","notes":"","initializers":[{"id":1,"locale_regex":"UnitedProvinces.Beaches","group":"*","compartment":"Ipre","value":"100"}]},"groups":[{"name":"Children","description":"","locales":[{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.255},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.255},{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.255}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[0,19]}},{"name":"Adults","description":"","locales":[{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.681},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.681},{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.681}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[20,49]}},{"name":"Seniors","description":"","locales":[{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.063},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.063},{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.063}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[50,84]}}],"group_specifications":[],"groups_locales_parameters":[{"id":0,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.27"},{"id":1,"param":"beta","locale":"UnitedProvinces.Hills","group":"Children","value":"0.27"},{"id":2,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.27"},{"id":3,"param":"sigma","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.21739"},{"id":4,"param":"sigma","locale":"UnitedProvinces.Hills","group":"Children","value":"0.21739"},{"id":5,"param":"sigma","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.21739"},{"id":6,"param":"psym","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.63658"},{"id":7,"param":"psym","locale":"UnitedProvinces.Hills","group":"Children","value":"0.63658"},{"id":8,"param":"psym","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.63658"},{"id":9,"param":"psev","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.04865"},{"id":10,"param":"psev","locale":"UnitedProvinces.Hills","group":"Children","value":"0.04865"},{"id":11,"param":"psev","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.04865"},{"id":12,"param":"pcri","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.08557"},{"id":13,"param":"pcri","locale":"UnitedProvinces.Hills","group":"Children","value":"0.08557"},{"id":14,"param":"pcri","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.08557"},{"id":15,"param":"pdeath","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.75472"},{"id":16,"param":"pdeath","locale":"UnitedProvinces.Hills","group":"Children","value":"0.75472"},{"id":17,"param":"pdeath","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.75472"},{"id":18,"param":"ra","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.05"},{"id":19,"param":"ra","locale":"UnitedProvinces.Hills","group":"Children","value":"0.05"},{"id":20,"param":"ra","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.05"},{"id":21,"param":"ri","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.75188"},{"id":22,"param":"ri","locale":"UnitedProvinces.Hills","group":"Children","value":"0.75188"},{"id":23,"param":"ri","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.75188"},{"id":24,"param":"rm","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.125"},{"id":25,"param":"rm","locale":"UnitedProvinces.Hills","group":"Children","value":"0.125"},{"id":26,"param":"rm","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.125"},{"id":27,"param":"rs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.07143"},{"id":28,"param":"rs","locale":"UnitedProvinces.Hills","group":"Children","value":"0.07143"},{"id":29,"param":"rs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.07143"},{"id":30,"param":"rc","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.07143"},{"id":31,"param":"rc","locale":"UnitedProvinces.Hills","group":"Children","value":"0.07143"},{"id":32,"param":"rc","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.07143"},{"id":33,"param":"is","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":34,"param":"is","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":35,"param":"is","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":36,"param":"iqs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":37,"param":"iqs","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":38,"param":"iqs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":39,"param":"ie","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":40,"param":"ie","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":41,"param":"ie","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":42,"param":"ipre","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":43,"param":"ipre","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":44,"param":"ipre","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":45,"param":"iasym","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":46,"param":"iasym","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":47,"param":"iasym","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":48,"param":"imild","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":49,"param":"imild","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":50,"param":"imild","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":51,"param":"vs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":52,"param":"vs","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":53,"param":"vs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":54,"param":"vqs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":55,"param":"vqs","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":56,"param":"vqs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":57,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.0333"},{"id":58,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Children","value":"0.0333"},{"id":59,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.0333"},{"id":60,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.27"},{"id":61,"param":"beta","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.27"},{"id":62,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.27"},{"id":63,"param":"sigma","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.21739"},{"id":64,"param":"sigma","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.21739"},{"id":65,"param":"sigma","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.21739"},{"id":66,"param":"psym","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.63658"},{"id":67,"param":"psym","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.63658"},{"id":68,"param":"psym","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.63658"},{"id":69,"param":"psev","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.04865"},{"id":70,"param":"psev","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.04865"},{"id":71,"param":"psev","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.04865"},{"id":72,"param":"pcri","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.08557"},{"id":73,"param":"pcri","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.08557"},{"id":74,"param":"pcri","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.08557"},{"id":75,"param":"pdeath","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.75472"},{"id":76,"param":"pdeath","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.75472"},{"id":77,"param":"pdeath","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.75472"},{"id":78,"param":"ra","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.05"},{"id":79,"param":"ra","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.05"},{"id":80,"param":"ra","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.05"},{"id":81,"param":"ri","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.75188"},{"id":82,"param":"ri","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.75188"},{"id":83,"param":"ri","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.75188"},{"id":84,"param":"rm","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.125"},{"id":85,"param":"rm","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.125"},{"id":86,"param":"rm","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.125"},{"id":87,"param":"rs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.07143"},{"id":88,"param":"rs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.07143"},{"id":89,"param":"rs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.07143"},{"id":90,"param":"rc","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.07143"},{"id":91,"param":"rc","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.07143"},{"id":92,"param":"rc","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.07143"},{"id":93,"param":"is","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":94,"param":"is","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":95,"param":"is","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":96,"param":"iqs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":97,"param":"iqs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":98,"param":"iqs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":99,"param":"ie","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":100,"param":"ie","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":101,"param":"ie","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":102,"param":"ipre","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":103,"param":"ipre","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":104,"param":"ipre","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":105,"param":"iasym","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":106,"param":"iasym","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":107,"param":"iasym","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":108,"param":"imild","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":109,"param":"imild","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":110,"param":"imild","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":111,"param":"vs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":112,"param":"vs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":113,"param":"vs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":114,"param":"vqs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":115,"param":"vqs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":116,"param":"vqs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":117,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.0333"},{"id":118,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.0333"},{"id":119,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.0333"},{"id":120,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.27"},{"id":121,"param":"beta","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.27"},{"id":122,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.27"},{"id":123,"param":"sigma","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.21739"},{"id":124,"param":"sigma","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.21739"},{"id":125,"param":"sigma","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.21739"},{"id":126,"param":"psym","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.63658"},{"id":127,"param":"psym","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.63658"},{"id":128,"param":"psym","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.63658"},{"id":129,"param":"psev","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.04865"},{"id":130,"param":"psev","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.04865"},{"id":131,"param":"psev","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.04865"},{"id":132,"param":"pcri","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.08557"},{"id":133,"param":"pcri","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.08557"},{"id":134,"param":"pcri","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.08557"},{"id":135,"param":"pdeath","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.75472"},{"id":136,"param":"pdeath","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.75472"},{"id":137,"param":"pdeath","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.75472"},{"id":138,"param":"ra","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.05"},{"id":139,"param":"ra","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.05"},{"id":140,"param":"ra","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.05"},{"id":141,"param":"ri","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.75188"},{"id":142,"param":"ri","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.75188"},{"id":143,"param":"ri","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.75188"},{"id":144,"param":"rm","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.125"},{"id":145,"param":"rm","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.125"},{"id":146,"param":"rm","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.125"},{"id":147,"param":"rs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.07143"},{"id":148,"param":"rs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.07143"},{"id":149,"param":"rs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.07143"},{"id":150,"param":"rc","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.07143"},{"id":151,"param":"rc","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.07143"},{"id":152,"param":"rc","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.07143"},{"id":153,"param":"is","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":154,"param":"is","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":155,"param":"is","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":156,"param":"iqs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":157,"param":"iqs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":158,"param":"iqs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":159,"param":"ie","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":160,"param":"ie","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":161,"param":"ie","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":162,"param":"ipre","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":163,"param":"ipre","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":164,"param":"ipre","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":165,"param":"iasym","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":166,"param":"iasym","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":167,"param":"iasym","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":168,"param":"imild","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":169,"param":"imild","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":170,"param":"imild","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":171,"param":"vs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":172,"param":"vs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":173,"param":"vs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":174,"param":"vqs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":175,"param":"vqs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":176,"param":"vqs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":177,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.0333"},{"id":178,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.0333"},{"id":179,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.0333"}],"facilities":[{"id":1,"name":"Household","description":"The household facility represents the pairwise connections between household members. Unlike schools and workplaces, everyone must be assigned to a household."},{"id":2,"name":"School","description":"The school facility represents all of the pairwise connections between people in schools, including both students and teachers. The current methods in SynthPops treat student and worker status as mutually exclusive."},{"id":3,"name":"Workplace","description":"The workplace facility represents all of the pairwise connections between people in workplaces, except for teachers working in schools. After some workers are assigned to the school contact layer as teachers, all remaining workers are assigned to workplaces. Workplaces are special in that there is little/no age structure so workers of all ages may be present in every workplace."},{"id":4,"name":"Community","description":"The community facility reflects the nature of contacts in shared public spaces like parks and recreational spaces, shopping centres, community centres, and public transportation. All links between individuals are considered undirected to reflect the ability of either individual in the pair to infect each other."}],"facilities_interactions":[{"locales":"UnitedProvinces.*","facilities":[[[0.37,0.45,0.18],[0.34,0.51,0.15],[0.26,0.29,0.45]],[[0.92,0.08,0.01],[0.65,0.35,0],[0.78,0.22,0]],[[0,0.64,0.36],[0.03,0.75,0.22],[0.04,0.69,0.27]],[[0.54,0.33,0.13],[0.15,0.61,0.24],[0.11,0.43,0.46]]]}],"facilities_timespent":[{"locales":"UnitedProvinces.*","matrix":[[0.4,0.4,0.46],[0.31,0.09,0.03],[0.08,0.31,0.27],[0.2,0.2,0.23]]}],"interaction_specifications":[],"schedules":[{"id":"5","name":"Vaccination","notes":"describe schedule for Vaccination...","detail":[{"id":1,"start_date":"2021-03-01","end_date":"2021-06-30","control_params":[{"name":"degree","value":"0"},{"name":"max_capacity","value":"5000"},{"name":"price_per_dose","value":"40"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"3","name":"Masks","notes":"describe schedule for Masks...","detail":[{"id":1,"start_date":"2021-03-01","end_date":"2021-06-30","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null},{"id":2,"start_date":"2021-07-01","end_date":"2021-12-31","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]}],"references":[],"mobility":[],"mobility_specifications":[],"border":{"data":[{"src_locale":"UnitedProvinces.Beaches","dst_locale":"UnitedProvinces.Hills","src_locale_id":"UP.Beaches","dst_locale_id":"UP.Hills","group":"*","value":0.7},{"src_locale":"UnitedProvinces.Beaches","dst_locale":"UnitedProvinces.Pastures","src_locale_id":"UP.Beaches","dst_locale_id":"UP.Pastures","group":"*","value":0.3},{"src_locale":"UnitedProvinces.Hills","dst_locale":"UnitedProvinces.Beaches","src_locale_id":"UP.Hills","dst_locale_id":"UP.Beaches","group":"*","value":0.6},{"src_locale":"UnitedProvinces.Pastures","dst_locale":"UnitedProvinces.Beaches","src_locale_id":"UP.Pastures","dst_locale_id":"UP.Beaches","group":"*","value":0.4}],"specifications":[{"id":1,"src_locale":"UnitedProvinces.*","dst_locale":"UnitedProvinces.*","group":"*","value":0,"impedance":70,"mobility_source":"GPWv4"}]},"airport":{"data":[],"specifications":[]},"facility":{"data":[],"specifications":[]},"optimize":{"configs":{"max_cost":1000000000000},"interventions":[{"name":"Vaccination","control_params":[{"name":"degree","min_value":0,"max_value":1},{"name":"max_capacity","min_value":9200,"max_value":9200},{"name":"price_per_dose","min_value":40,"max_value":40}]},{"name":"Masks","control_params":[{"name":"compliance","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":0.05,"max_value":0.05},{"name":"max_transmission_reduction","min_value":0.5,"max_value":0.5}]}]}}'''

COVID_B = r'''{"features":{"data_source":"GPWv4","region":"UP","region_name":"UnitedProvinces","level":1,"year":2020,"start_date":"2021-01-01","end_date":"2021-12-31"},"interventions":[{"name":"Vaccination","notes":"Each day, vaccination directly moves a fixed number of people from susceptible compartment to vaccinated compartment. The number of people vaccinated is determined by the product of 'degree' and 'max_capacity'. Hence, when the 'degree' is 100%, 'max_capacity' people are vaccinated.","control_params":[{"id":"1","name":"degree","description":"0% means no vaccination at all while 100% achieves the max_capacity per day","default_value":"0"},{"id":"2","name":"max_capacity","description":"Maximum number of people vaccinated per day if 100% degree is reached","default_value":"50000"},{"id":"3","name":"price_per_dose","description":"Price per dose (or 2-dose if the vaccine requires 2 doses)","default_value":"40"}],"effect":"def effect(cp, locales):\n\tsim.move({'compartment':'S', 'locale':locales, 'group':'*'}, {'compartment':'V','locale':locales, 'group':'*'}, cp['degree']*cp['max_capacity'])","cost":"def cost(cp, locales):\n\tdoses = cp['degree']*cp['max_capacity']\n\tsim.add({'locale':locales}, doses*cp['price_per_dose'])","id":"5"},{"name":"Masks","notes":"Mask reduces the transmission rate in all facilities except Household (since people do not where mask in household). Mask is modeled by two control parameters 'compliance' and 'max_transmission_reduction'. At 100% 'compliance', there will be a 'max_transmission_reduction' to the transmission rate in all facilities except Household.","control_params":[{"id":"1","name":"compliance","description":"Percentage of people wearing mask","default_value":"0"},{"id":"2","name":"cost_per_day","description":"Cost per mask","default_value":"0.05"},{"id":"3","name":"max_transmission_reduction","description":"Maximum percentage of reduction for transmission rate when there is 100% compliance","default_value":"0.5"}],"effect":"def effect(cp, locales):\n\tsim.apply({'parameter':'beta', 'facility':'~Household', 'locale':locales}, 1-cp['compliance']*cp['max_transmission_reduction'])","cost":"def cost(cp, locales):\n\tcompliance_count = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['compliance']\n\tsim.add({'locale':locales}, compliance_count*cp['cost_per_day'])","id":"3"},{"id":"6","name":"School closure","notes":"","control_params":[{"id":"1","name":"percentage","description":"describe parameter 1 here...","default_value":"0"},{"id":"2","name":"cost_per_day","description":"describe parameter 2 here...","default_value":"1.8"}],"effect":"def effect(cp, locales):\n    sim.apply({\"facility\": \"School\", \"locale\":locales, \"group-from\":\"*\", \"group-to\":\"*\"}, 1-cp['percentage'])\n","cost":"def cost(cp, locales):\n    affectedCount = sim.select({'compartment':'*', 'locale':locales, 'group':\"Children\"})['Value'].sum() * cp['percentage']\n    sim.add({'locale':locales}, affectedCount*cp['cost_per_day'])\n"},{"id":"7","name":"Workplace closure","notes":"","control_params":[{"id":"1","name":"percentage","description":"describe parameter 1 here...","default_value":"0"},{"id":"2","name":"cost_per_day","description":"describe parameter 2 here...","default_value":"1.8"}],"effect":"def effect(cp, locales):\n    sim.apply({\"facility\": \"Workplace\", \"locale\":locales, \"group-from\":\"*\", \"group-to\":\"*\"}, 1-cp['percentage'])\n","cost":"def cost(cp, locales):\n    affectedCount = sim.select({'compartment':'*', 'locale':locales, 'group':\"Adults\"})['Value'].sum() * cp['percentage']\n    sim.add({'locale':locales}, affectedCount*cp['cost_per_day'])\n"}],"costs":[{"id":"1","name":"Infectious_cost","notes":"","intervention":null,"control_params":[{"id":"1","name":"cost_per_day","description":"describe parameter 1 here...","default_value":"173"}],"func":"def cost(cp):\n    infectious_count = sim.select({'compartment':'{\"tag\":\"infectious\"}'})['Value'].sum()\n    sim.add({\"intervention\":\"Infectious_cost\"}, infectious_count*cp['cost_per_day'])"},{"id":"2","name":"Hospitalized_cost","notes":"","intervention":null,"control_params":[{"id":"1","name":"cost_per_day","description":"describe parameter 1 here...","default_value":"250"}],"func":"def cost(cp):\n    hospitalized_count = sim.select({'compartment':'{\"tag\":\"hospitalized\"}'})['Value'].sum()\n    sim.add({\"intervention\":\"Hospitalized_cost\"}, hospitalized_count*cp['cost_per_day'])"},{"id":"3","name":"Death_cost","notes":"","intervention":null,"control_params":[{"id":"1","name":"cost_per_death","description":"describe parameter 1 here...","default_value":"100000"}],"func":"def cost(cp):\n    death_increase = sim.select({'compartment':'D', 'value-mode':'change'})['Value'].sum()\n    sim.add({\"intervention\":\"Death_cost\"}, death_increase*cp['cost_per_death'])"}],"locales":[{"population":1012394,"area":99324,"name":"UnitedProvinces","id":"UP","parent_id":""},{"population":496073.06,"area":30201,"name":"UnitedProvinces.Beaches","id":"UP.Beaches","parent_id":"UP"},{"population":313842.14,"area":24504,"name":"UnitedProvinces.Hills","id":"UP.Hills","parent_id":"UP"},{"population":202478.8,"area":44619,"name":"UnitedProvinces.Pastures","id":"UP.Pastures","parent_id":"UP"}],"model":{"name":"COVID19-Model","compartments":[{"id":1,"name":"S","desc":"Susceptible","equation":"gamma*R - vs*S + iqs*Qs - is*S*(Iasym + Ipre + Imild + Qasym + Qpre + Hmild + Hsev + Hcri) / N - beta * (Iasym + Ipre + Imild) * S / N - 0.2 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * S / N","tags":["susceptible"]},{"id":2,"name":"E","desc":"Exposed","equation":"-(ie*E) -(1-psym)*sigma*E -(psym*sigma*E) + beta * (Iasym + Ipre + Imild) * S / N + 0.2 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * S / N","tags":["exposed"]},{"id":3,"name":"Ipre","desc":"Presymptomatic","equation":"psym*sigma*E-ipre*Ipre-ri*Ipre","tags":["infectious"]},{"id":4,"name":"Iasym","desc":"Asymptomatic","equation":"(1-psym)*sigma*E -iasym*Iasym -ra*Iasym","tags":["infectious"]},{"id":5,"name":"Imild","desc":"Mild","equation":"ri*Ipre -imild*Imild -psev*rm*Imild -(1-psev)*rm*Imild","tags":["infectious"]},{"id":6,"name":"Hsev","desc":"Severe with Hospitalization","equation":"psev*rm*Imild + 0.5*psev*rm*Hmild -pcri*rs*Hsev -(1-pcri)*rs*Hsev","tags":["infectious","hospitalized"]},{"id":7,"name":"Hcri","desc":"Critical with Hospitalization","equation":"pcri*rs*Hsev -pdeath*rc*Hcri -(1-pdeath)*rc*Hcri","tags":["infectious","hospitalized"]},{"id":8,"name":"D","desc":"Death","equation":"pdeath*rc*Hcri","tags":["death"]},{"id":9,"name":"R","desc":"Recovered","equation":"-(gamma*R) + ra*Iasym + (1-psev)*rm*Imild + ra*Qasym + (1-0.5*psev)*rm*Hmild + (1-pcri)*rs*Hsev + (1-pdeath)*rc*Hcri","tags":["recovered"]},{"id":10,"name":"Qs","desc":"Quarantined susceptible","equation":"-vqs*Qs + is*S*(Iasym + Ipre + Imild + Qasym + Qpre + Hmild + Hsev + Hcri) / N -iqs*Qs - 0.12 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * Qs / N - 0.6 * beta * (Iasym + Ipre + Imild) * Qs / N","tags":["susceptible","isolated"]},{"id":11,"name":"Qe","desc":"Quarantined exposed","equation":"ie*E -psym*sigma*Qe -(1-psym)*sigma*Qe + 0.12 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * Qs / N + 0.6 * beta * (Iasym + Ipre + Imild) * Qs / N","tags":["exposed","isolated"]},{"id":12,"name":"Qpre","desc":"Isolated presymptomatic","equation":"psym*sigma*Qe + ipre*Ipre -ri*Qpre","tags":["infectious","isolated"]},{"id":13,"name":"Qasym","desc":"Isolated asymptomatic","equation":"(1-psym)*sigma*Qe + iasym*Iasym -ra*Qasym","tags":["infectious","isolated"]},{"id":14,"name":"Hmild","desc":"Hospitalized mild","equation":"ri*Qpre + imild*Imild -0.5*psev*rm*Hmild -(1-0.5*psev)*rm*Hmild","tags":["infectious","hospitalized"]},{"id":15,"name":"V","desc":"Vaccinated","equation":"vs*S+vqs*Qs","tags":["vaccinated"]}],"parameters":[{"id":1,"name":"beta","desc":"Transmission rate","default_value":"0.27","tags":["transmission"]},{"id":2,"name":"sigma","desc":"Incubation rate","default_value":"0.21739","tags":[]},{"id":3,"name":"psym","desc":"Probability of having symptoms","default_value":"0.63658","tags":[]},{"id":4,"name":"psev","desc":"Probability of developing serious complications","default_value":"0.04865","tags":[]},{"id":5,"name":"pcri","desc":"Probability of developing critical complications","default_value":"0.08557","tags":[]},{"id":6,"name":"pdeath","desc":"Probability of death","default_value":"0.75472","tags":[]},{"id":7,"name":"ra","desc":"Asymptomatic recovery rate","default_value":"0.05","tags":[]},{"id":8,"name":"ri","desc":"Viral shedding rate","default_value":"0.75188","tags":[]},{"id":9,"name":"rm","desc":"Mild recovery rate","default_value":"0.125","tags":[]},{"id":10,"name":"rs","desc":"Severe recovery rate","default_value":"0.07143","tags":[]},{"id":11,"name":"rc","desc":"Critical recovery rate","default_value":"0.07143","tags":[]},{"id":12,"name":"is","desc":"Susceptible quarantine rate","default_value":"0.02643","tags":["quarantine"]},{"id":13,"name":"iqs","desc":"Susceptible end-of-quarantine rate","default_value":"0.07143","tags":[]},{"id":14,"name":"ie","desc":"Exposed quarantine rate","default_value":"0.09301","tags":["quarantine"]},{"id":15,"name":"ipre","desc":"Presymptomatic quarantine rate","default_value":"0.16179","tags":["quarantine"]},{"id":16,"name":"iasym","desc":"Asymptomatic quarantine rate","default_value":"0.16179","tags":["quarantine"]},{"id":17,"name":"imild","desc":"Mild hospitalized rate","default_value":"0.19231","tags":[]},{"id":19,"name":"vs","desc":"Daily percentage of susceptible being vaccinated","default_value":"0","tags":[]},{"id":20,"name":"vqs","desc":"Daily percentage of quarantine-susceptible being vaccinated","default_value":"0","tags":[]},{"id":21,"name":"gamma","desc":"Immunity-loss rate","default_value":"0.0333","tags":[]}]},"initial_info":{"name":"UpB1","notes":"","initializers":[{"id":1,"locale_regex":"UnitedProvinces.Beaches","group":"*","compartment":"Ipre","value":"100"}]},"groups":[{"name":"Children","description":"","locales":[{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.255},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.255},{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.255}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[0,19]}},{"name":"Adults","description":"","locales":[{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.681},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.681},{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.681}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[20,49]}},{"name":"Seniors","description":"","locales":[{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.063},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.063},{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.063}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[50,84]}}],"group_specifications":[],"groups_locales_parameters":[{"id":0,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.27"},{"id":1,"param":"beta","locale":"UnitedProvinces.Hills","group":"Children","value":"0.27"},{"id":2,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.27"},{"id":3,"param":"sigma","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.21739"},{"id":4,"param":"sigma","locale":"UnitedProvinces.Hills","group":"Children","value":"0.21739"},{"id":5,"param":"sigma","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.21739"},{"id":6,"param":"psym","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.63658"},{"id":7,"param":"psym","locale":"UnitedProvinces.Hills","group":"Children","value":"0.63658"},{"id":8,"param":"psym","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.63658"},{"id":9,"param":"psev","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.04865"},{"id":10,"param":"psev","locale":"UnitedProvinces.Hills","group":"Children","value":"0.04865"},{"id":11,"param":"psev","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.04865"},{"id":12,"param":"pcri","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.08557"},{"id":13,"param":"pcri","locale":"UnitedProvinces.Hills","group":"Children","value":"0.08557"},{"id":14,"param":"pcri","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.08557"},{"id":15,"param":"pdeath","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.75472"},{"id":16,"param":"pdeath","locale":"UnitedProvinces.Hills","group":"Children","value":"0.75472"},{"id":17,"param":"pdeath","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.75472"},{"id":18,"param":"ra","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.05"},{"id":19,"param":"ra","locale":"UnitedProvinces.Hills","group":"Children","value":"0.05"},{"id":20,"param":"ra","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.05"},{"id":21,"param":"ri","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.75188"},{"id":22,"param":"ri","locale":"UnitedProvinces.Hills","group":"Children","value":"0.75188"},{"id":23,"param":"ri","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.75188"},{"id":24,"param":"rm","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.125"},{"id":25,"param":"rm","locale":"UnitedProvinces.Hills","group":"Children","value":"0.125"},{"id":26,"param":"rm","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.125"},{"id":27,"param":"rs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.07143"},{"id":28,"param":"rs","locale":"UnitedProvinces.Hills","group":"Children","value":"0.07143"},{"id":29,"param":"rs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.07143"},{"id":30,"param":"rc","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.07143"},{"id":31,"param":"rc","locale":"UnitedProvinces.Hills","group":"Children","value":"0.07143"},{"id":32,"param":"rc","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.07143"},{"id":33,"param":"is","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":34,"param":"is","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":35,"param":"is","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":36,"param":"iqs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":37,"param":"iqs","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":38,"param":"iqs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":39,"param":"ie","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":40,"param":"ie","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":41,"param":"ie","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":42,"param":"ipre","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":43,"param":"ipre","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":44,"param":"ipre","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":45,"param":"iasym","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":46,"param":"iasym","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":47,"param":"iasym","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":48,"param":"imild","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":49,"param":"imild","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":50,"param":"imild","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":51,"param":"vs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":52,"param":"vs","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":53,"param":"vs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":54,"param":"vqs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":55,"param":"vqs","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":56,"param":"vqs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":57,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.0333"},{"id":58,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Children","value":"0.0333"},{"id":59,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.0333"},{"id":60,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.27"},{"id":61,"param":"beta","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.27"},{"id":62,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.27"},{"id":63,"param":"sigma","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.21739"},{"id":64,"param":"sigma","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.21739"},{"id":65,"param":"sigma","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.21739"},{"id":66,"param":"psym","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.63658"},{"id":67,"param":"psym","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.63658"},{"id":68,"param":"psym","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.63658"},{"id":69,"param":"psev","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.04865"},{"id":70,"param":"psev","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.04865"},{"id":71,"param":"psev","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.04865"},{"id":72,"param":"pcri","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.08557"},{"id":73,"param":"pcri","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.08557"},{"id":74,"param":"pcri","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.08557"},{"id":75,"param":"pdeath","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.75472"},{"id":76,"param":"pdeath","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.75472"},{"id":77,"param":"pdeath","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.75472"},{"id":78,"param":"ra","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.05"},{"id":79,"param":"ra","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.05"},{"id":80,"param":"ra","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.05"},{"id":81,"param":"ri","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.75188"},{"id":82,"param":"ri","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.75188"},{"id":83,"param":"ri","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.75188"},{"id":84,"param":"rm","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.125"},{"id":85,"param":"rm","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.125"},{"id":86,"param":"rm","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.125"},{"id":87,"param":"rs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.07143"},{"id":88,"param":"rs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.07143"},{"id":89,"param":"rs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.07143"},{"id":90,"param":"rc","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.07143"},{"id":91,"param":"rc","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.07143"},{"id":92,"param":"rc","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.07143"},{"id":93,"param":"is","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":94,"param":"is","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":95,"param":"is","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":96,"param":"iqs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":97,"param":"iqs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":98,"param":"iqs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":99,"param":"ie","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":100,"param":"ie","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":101,"param":"ie","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":102,"param":"ipre","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":103,"param":"ipre","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":104,"param":"ipre","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":105,"param":"iasym","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":106,"param":"iasym","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":107,"param":"iasym","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":108,"param":"imild","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":109,"param":"imild","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":110,"param":"imild","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":111,"param":"vs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":112,"param":"vs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":113,"param":"vs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":114,"param":"vqs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":115,"param":"vqs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":116,"param":"vqs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":117,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.0333"},{"id":118,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.0333"},{"id":119,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.0333"},{"id":120,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.27"},{"id":121,"param":"beta","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.27"},{"id":122,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.27"},{"id":123,"param":"sigma","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.21739"},{"id":124,"param":"sigma","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.21739"},{"id":125,"param":"sigma","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.21739"},{"id":126,"param":"psym","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.63658"},{"id":127,"param":"psym","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.63658"},{"id":128,"param":"psym","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.63658"},{"id":129,"param":"psev","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.04865"},{"id":130,"param":"psev","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.04865"},{"id":131,"param":"psev","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.04865"},{"id":132,"param":"pcri","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.08557"},{"id":133,"param":"pcri","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.08557"},{"id":134,"param":"pcri","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.08557"},{"id":135,"param":"pdeath","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.75472"},{"id":136,"param":"pdeath","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.75472"},{"id":137,"param":"pdeath","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.75472"},{"id":138,"param":"ra","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.05"},{"id":139,"param":"ra","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.05"},{"id":140,"param":"ra","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.05"},{"id":141,"param":"ri","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.75188"},{"id":142,"param":"ri","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.75188"},{"id":143,"param":"ri","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.75188"},{"id":144,"param":"rm","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.125"},{"id":145,"param":"rm","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.125"},{"id":146,"param":"rm","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.125"},{"id":147,"param":"rs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.07143"},{"id":148,"param":"rs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.07143"},{"id":149,"param":"rs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.07143"},{"id":150,"param":"rc","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.07143"},{"id":151,"param":"rc","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.07143"},{"id":152,"param":"rc","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.07143"},{"id":153,"param":"is","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":154,"param":"is","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":155,"param":"is","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":156,"param":"iqs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":157,"param":"iqs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":158,"param":"iqs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":159,"param":"ie","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":160,"param":"ie","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":161,"param":"ie","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":162,"param":"ipre","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":163,"param":"ipre","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":164,"param":"ipre","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":165,"param":"iasym","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":166,"param":"iasym","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":167,"param":"iasym","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":168,"param":"imild","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":169,"param":"imild","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":170,"param":"imild","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":171,"param":"vs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":172,"param":"vs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":173,"param":"vs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":174,"param":"vqs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":175,"param":"vqs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":176,"param":"vqs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":177,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.0333"},{"id":178,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.0333"},{"id":179,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.0333"}],"facilities":[{"id":1,"name":"Household","description":"The household facility represents the pairwise connections between household members. Unlike schools and workplaces, everyone must be assigned to a household."},{"id":2,"name":"School","description":"The school facility represents all of the pairwise connections between people in schools, including both students and teachers. The current methods in SynthPops treat student and worker status as mutually exclusive."},{"id":3,"name":"Workplace","description":"The workplace facility represents all of the pairwise connections between people in workplaces, except for teachers working in schools. After some workers are assigned to the school contact layer as teachers, all remaining workers are assigned to workplaces. Workplaces are special in that there is little/no age structure so workers of all ages may be present in every workplace."},{"id":4,"name":"Community","description":"The community facility reflects the nature of contacts in shared public spaces like parks and recreational spaces, shopping centres, community centres, and public transportation. All links between individuals are considered undirected to reflect the ability of either individual in the pair to infect each other."}],"facilities_interactions":[{"locales":"UnitedProvinces.*","facilities":[[[0.37,0.45,0.18],[0.34,0.51,0.15],[0.26,0.29,0.45]],[[0.92,0.08,0.01],[0.65,0.35,0],[0.78,0.22,0]],[[0,0.64,0.36],[0.03,0.75,0.22],[0.04,0.69,0.27]],[[0.54,0.33,0.13],[0.15,0.61,0.24],[0.11,0.43,0.46]]]}],"facilities_timespent":[{"locales":"UnitedProvinces.*","matrix":[[0.4,0.4,0.46],[0.31,0.09,0.03],[0.08,0.31,0.27],[0.2,0.2,0.23]]}],"interaction_specifications":[],"schedules":[{"id":"5","name":"Vaccination","notes":"describe schedule for Vaccination...","detail":[{"id":1,"start_date":"2021-03-01","end_date":"2021-06-30","control_params":[{"name":"degree","value":"0"},{"name":"max_capacity","value":"5000"},{"name":"price_per_dose","value":"40"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"3","name":"Masks","notes":"describe schedule for Masks...","detail":[{"id":1,"start_date":"2021-03-01","end_date":"2021-06-30","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null},{"id":2,"start_date":"2021-07-01","end_date":"2021-12-31","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"6","name":"School closure","notes":"describe schedule for School closure...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"percentage","value":"0.2"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"7","name":"Workplace closure","notes":"describe schedule for Workplace closure...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"percentage","value":"0.2"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]}],"references":[],"mobility":[],"mobility_specifications":[],"border":{"data":[{"src_locale":"UnitedProvinces.Beaches","dst_locale":"UnitedProvinces.Hills","src_locale_id":"UP.Beaches","dst_locale_id":"UP.Hills","group":"*","value":0.7},{"src_locale":"UnitedProvinces.Beaches","dst_locale":"UnitedProvinces.Pastures","src_locale_id":"UP.Beaches","dst_locale_id":"UP.Pastures","group":"*","value":0.3},{"src_locale":"UnitedProvinces.Hills","dst_locale":"UnitedProvinces.Beaches","src_locale_id":"UP.Hills","dst_locale_id":"UP.Beaches","group":"*","value":0.6},{"src_locale":"UnitedProvinces.Pastures","dst_locale":"UnitedProvinces.Beaches","src_locale_id":"UP.Pastures","dst_locale_id":"UP.Beaches","group":"*","value":0.4}],"specifications":[{"id":1,"src_locale":"UnitedProvinces.*","dst_locale":"UnitedProvinces.*","group":"*","value":0,"impedance":70,"mobility_source":"GPWv4"}]},"airport":{"data":[],"specifications":[]},"facility":{"data":[],"specifications":[]},"optimize":{"configs":{"max_cost":1000000000000},"interventions":[{"name":"Vaccination","control_params":[{"name":"degree","min_value":0,"max_value":1},{"name":"max_capacity","min_value":9200,"max_value":9200},{"name":"price_per_dose","min_value":40,"max_value":40}]},{"name":"Masks","control_params":[{"name":"compliance","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":0.05,"max_value":0.05},{"name":"max_transmission_reduction","min_value":0.5,"max_value":0.5}]},{"name":"School closure","control_params":[{"name":"percentage","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]},{"name":"Workplace closure","control_params":[{"name":"percentage","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]}]}}'''

COVID_C = r'''{"features":{"data_source":"GPWv4","region":"UP","region_name":"UnitedProvinces","level":1,"year":2020,"start_date":"2021-01-01","end_date":"2021-12-31"},"interventions":[{"name":"Vaccination","notes":"Each day, vaccination directly moves a fixed number of people from susceptible compartment to vaccinated compartment. The number of people vaccinated is determined by the product of 'degree' and 'max_capacity'. Hence, when the 'degree' is 100%, 'max_capacity' people are vaccinated.","control_params":[{"id":"1","name":"degree","description":"0% means no vaccination at all while 100% achieves the max_capacity per day","default_value":"0"},{"id":"2","name":"max_capacity","description":"Maximum number of people vaccinated per day if 100% degree is reached","default_value":"50000"},{"id":"3","name":"price_per_dose","description":"Price per dose (or 2-dose if the vaccine requires 2 doses)","default_value":"40"}],"effect":"def effect(cp, locales):\n\tsim.move({'compartment':'S', 'locale':locales, 'group':'*'}, {'compartment':'V','locale':locales, 'group':'*'}, cp['degree']*cp['max_capacity'])","cost":"def cost(cp, locales):\n\tdoses = cp['degree']*cp['max_capacity']\n\tsim.add({'locale':locales}, doses*cp['price_per_dose'])","id":"5"},{"name":"Masks","notes":"Mask reduces the transmission rate in all facilities except Household (since people do not where mask in household). Mask is modeled by two control parameters 'compliance' and 'max_transmission_reduction'. At 100% 'compliance', there will be a 'max_transmission_reduction' to the transmission rate in all facilities except Household.","control_params":[{"id":"1","name":"compliance","description":"Percentage of people wearing mask","default_value":"0"},{"id":"2","name":"cost_per_day","description":"Cost per mask","default_value":"0.05"},{"id":"3","name":"max_transmission_reduction","description":"Maximum percentage of reduction for transmission rate when there is 100% compliance","default_value":"0.5"}],"effect":"def effect(cp, locales):\n\tsim.apply({'parameter':'beta', 'facility':'~Household', 'locale':locales}, 1-cp['compliance']*cp['max_transmission_reduction'])","cost":"def cost(cp, locales):\n\tcompliance_count = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['compliance']\n\tsim.add({'locale':locales}, compliance_count*cp['cost_per_day'])","id":"3"},{"id":"6","name":"School closure","notes":"","control_params":[{"id":"1","name":"percentage","description":"describe parameter 1 here...","default_value":"0"},{"id":"2","name":"cost_per_day","description":"describe parameter 2 here...","default_value":"1.8"}],"effect":"def effect(cp, locales):\n    sim.apply({\"facility\": \"School\", \"locale\":locales, \"group-from\":\"*\", \"group-to\":\"*\"}, 1-cp['percentage'])\n","cost":"def cost(cp, locales):\n    affectedCount = sim.select({'compartment':'*', 'locale':locales, 'group':\"Children\"})['Value'].sum() * cp['percentage']\n    sim.add({'locale':locales}, affectedCount*cp['cost_per_day'])\n"},{"id":"7","name":"Workplace closure","notes":"","control_params":[{"id":"1","name":"percentage","description":"describe parameter 1 here...","default_value":"0"},{"id":"2","name":"cost_per_day","description":"describe parameter 2 here...","default_value":"1.8"}],"effect":"def effect(cp, locales):\n    sim.apply({\"facility\": \"Workplace\", \"locale\":locales, \"group-from\":\"*\", \"group-to\":\"*\"}, 1-cp['percentage'])\n","cost":"def cost(cp, locales):\n    affectedCount = sim.select({'compartment':'*', 'locale':locales, 'group':\"Adults\"})['Value'].sum() * cp['percentage']\n    sim.add({'locale':locales}, affectedCount*cp['cost_per_day'])\n"},{"name":"Mass_Screening_and_Contact_Tracing","notes":"Mass screen allows to detect positive cases while contact tracing allows to isolate/quarantine cases. We model this intervention by a single control parameter 'degree' that modifies the rate of transition of normal compartment to the respective quarantined/isolated compartment.","control_params":[{"id":"1","name":"degree","description":"Degree is between 0% and 100% where 0% is no isolation/quarantine while 100% is extreme measure","default_value":"0.4"},{"id":"2","name":"cost_per_day","description":"The cost to maintain this degree of intervention per person per day","default_value":"1.8"}],"effect":"def effect(cp, locales):\n\tsim.apply({'parameter':'{\"tag\":\"quarantine\"}', 'locale':locales}, cp['degree'])","cost":"def cost(cp, locales):\n\taffectedCount = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['degree']\n\tsim.add({'locale':locales}, affectedCount*cp['cost_per_day'])","id":"8"},{"name":"Border_Closures","notes":"Travel restriction restricts the time spent of population of one locale in every other locale and vice versa. For example, a 100% degree of travel restriction in one locale prevents population in that locale from spending time in every other locale and population from every other locale from spending time in that locale.","control_params":[{"id":"1","name":"degree","description":"Degree of travel restriction. 0% means no restriction at all.","default_value":"0.3"},{"id":"2","name":"cost_per_day","description":"Cost of maintaining the degree of travel restriction per person per day","default_value":"1.8"}],"effect":"def effect(cp, locales):\n\tsim.apply({'locale-from':locales, 'locale-to':'~'+locales}, 1-cp['degree'])\n\tsim.apply({'locale-from':'~'+locales, 'locale-to':locales}, 1-cp['degree'])","cost":"def cost(cp, locales):\n\taffectedCount = sim.select({'compartment':'*', 'locale':locales})['Value'].sum() * cp['degree']\n\tsim.add({'locale':locales}, affectedCount*cp['cost_per_day'])","id":"9"}],"costs":[{"id":"1","name":"Infectious_cost","notes":"","intervention":null,"control_params":[{"id":"1","name":"cost_per_day","description":"describe parameter 1 here...","default_value":"173"}],"func":"def cost(cp):\n    infectious_count = sim.select({'compartment':'{\"tag\":\"infectious\"}'})['Value'].sum()\n    sim.add({\"intervention\":\"Infectious_cost\"}, infectious_count*cp['cost_per_day'])"},{"id":"2","name":"Hospitalized_cost","notes":"","intervention":null,"control_params":[{"id":"1","name":"cost_per_day","description":"describe parameter 1 here...","default_value":"250"}],"func":"def cost(cp):\n    hospitalized_count = sim.select({'compartment':'{\"tag\":\"hospitalized\"}'})['Value'].sum()\n    sim.add({\"intervention\":\"Hospitalized_cost\"}, hospitalized_count*cp['cost_per_day'])"},{"id":"3","name":"Death_cost","notes":"","intervention":null,"control_params":[{"id":"1","name":"cost_per_death","description":"describe parameter 1 here...","default_value":"100000"}],"func":"def cost(cp):\n    death_increase = sim.select({'compartment':'D', 'value-mode':'change'})['Value'].sum()\n    sim.add({\"intervention\":\"Death_cost\"}, death_increase*cp['cost_per_death'])"}],"locales":[{"population":1012394,"area":99324,"name":"UnitedProvinces","id":"UP","parent_id":""},{"population":496073.06,"area":30201,"name":"UnitedProvinces.Beaches","id":"UP.Beaches","parent_id":"UP"},{"population":313842.14,"area":24504,"name":"UnitedProvinces.Hills","id":"UP.Hills","parent_id":"UP"},{"population":202478.8,"area":44619,"name":"UnitedProvinces.Pastures","id":"UP.Pastures","parent_id":"UP"}],"model":{"name":"COVID19-Model","compartments":[{"id":1,"name":"S","desc":"Susceptible","equation":"gamma*R - vs*S + iqs*Qs - is*S*(Iasym + Ipre + Imild + Qasym + Qpre + Hmild + Hsev + Hcri) / N - beta * (Iasym + Ipre + Imild) * S / N - 0.2 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * S / N","tags":["susceptible"]},{"id":2,"name":"E","desc":"Exposed","equation":"-(ie*E) -(1-psym)*sigma*E -(psym*sigma*E) + beta * (Iasym + Ipre + Imild) * S / N + 0.2 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * S / N","tags":["exposed"]},{"id":3,"name":"Ipre","desc":"Presymptomatic","equation":"psym*sigma*E-ipre*Ipre-ri*Ipre","tags":["infectious"]},{"id":4,"name":"Iasym","desc":"Asymptomatic","equation":"(1-psym)*sigma*E -iasym*Iasym -ra*Iasym","tags":["infectious"]},{"id":5,"name":"Imild","desc":"Mild","equation":"ri*Ipre -imild*Imild -psev*rm*Imild -(1-psev)*rm*Imild","tags":["infectious"]},{"id":6,"name":"Hsev","desc":"Severe with Hospitalization","equation":"psev*rm*Imild + 0.5*psev*rm*Hmild -pcri*rs*Hsev -(1-pcri)*rs*Hsev","tags":["infectious","hospitalized"]},{"id":7,"name":"Hcri","desc":"Critical with Hospitalization","equation":"pcri*rs*Hsev -pdeath*rc*Hcri -(1-pdeath)*rc*Hcri","tags":["infectious","hospitalized"]},{"id":8,"name":"D","desc":"Death","equation":"pdeath*rc*Hcri","tags":["death"]},{"id":9,"name":"R","desc":"Recovered","equation":"-(gamma*R) + ra*Iasym + (1-psev)*rm*Imild + ra*Qasym + (1-0.5*psev)*rm*Hmild + (1-pcri)*rs*Hsev + (1-pdeath)*rc*Hcri","tags":["recovered"]},{"id":10,"name":"Qs","desc":"Quarantined susceptible","equation":"-vqs*Qs + is*S*(Iasym + Ipre + Imild + Qasym + Qpre + Hmild + Hsev + Hcri) / N -iqs*Qs - 0.12 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * Qs / N - 0.6 * beta * (Iasym + Ipre + Imild) * Qs / N","tags":["susceptible","isolated"]},{"id":11,"name":"Qe","desc":"Quarantined exposed","equation":"ie*E -psym*sigma*Qe -(1-psym)*sigma*Qe + 0.12 * beta * (Qasym + Qpre + Hmild + Hsev + Hcri) * Qs / N + 0.6 * beta * (Iasym + Ipre + Imild) * Qs / N","tags":["exposed","isolated"]},{"id":12,"name":"Qpre","desc":"Isolated presymptomatic","equation":"psym*sigma*Qe + ipre*Ipre -ri*Qpre","tags":["infectious","isolated"]},{"id":13,"name":"Qasym","desc":"Isolated asymptomatic","equation":"(1-psym)*sigma*Qe + iasym*Iasym -ra*Qasym","tags":["infectious","isolated"]},{"id":14,"name":"Hmild","desc":"Hospitalized mild","equation":"ri*Qpre + imild*Imild -0.5*psev*rm*Hmild -(1-0.5*psev)*rm*Hmild","tags":["infectious","hospitalized"]},{"id":15,"name":"V","desc":"Vaccinated","equation":"vs*S+vqs*Qs","tags":["vaccinated"]}],"parameters":[{"id":1,"name":"beta","desc":"Transmission rate","default_value":"0.27","tags":["transmission"]},{"id":2,"name":"sigma","desc":"Incubation rate","default_value":"0.21739","tags":[]},{"id":3,"name":"psym","desc":"Probability of having symptoms","default_value":"0.63658","tags":[]},{"id":4,"name":"psev","desc":"Probability of developing serious complications","default_value":"0.04865","tags":[]},{"id":5,"name":"pcri","desc":"Probability of developing critical complications","default_value":"0.08557","tags":[]},{"id":6,"name":"pdeath","desc":"Probability of death","default_value":"0.75472","tags":[]},{"id":7,"name":"ra","desc":"Asymptomatic recovery rate","default_value":"0.05","tags":[]},{"id":8,"name":"ri","desc":"Viral shedding rate","default_value":"0.75188","tags":[]},{"id":9,"name":"rm","desc":"Mild recovery rate","default_value":"0.125","tags":[]},{"id":10,"name":"rs","desc":"Severe recovery rate","default_value":"0.07143","tags":[]},{"id":11,"name":"rc","desc":"Critical recovery rate","default_value":"0.07143","tags":[]},{"id":12,"name":"is","desc":"Susceptible quarantine rate","default_value":"0.02643","tags":["quarantine"]},{"id":13,"name":"iqs","desc":"Susceptible end-of-quarantine rate","default_value":"0.07143","tags":[]},{"id":14,"name":"ie","desc":"Exposed quarantine rate","default_value":"0.09301","tags":["quarantine"]},{"id":15,"name":"ipre","desc":"Presymptomatic quarantine rate","default_value":"0.16179","tags":["quarantine"]},{"id":16,"name":"iasym","desc":"Asymptomatic quarantine rate","default_value":"0.16179","tags":["quarantine"]},{"id":17,"name":"imild","desc":"Mild hospitalized rate","default_value":"0.19231","tags":[]},{"id":19,"name":"vs","desc":"Daily percentage of susceptible being vaccinated","default_value":"0","tags":[]},{"id":20,"name":"vqs","desc":"Daily percentage of quarantine-susceptible being vaccinated","default_value":"0","tags":[]},{"id":21,"name":"gamma","desc":"Immunity-loss rate","default_value":"0.0333","tags":[]}]},"initial_info":{"name":"UpC1","notes":"","initializers":[{"id":1,"locale_regex":"UnitedProvinces.Beaches","group":"*","compartment":"Ipre","value":"100"}]},"groups":[{"name":"Children","description":"","locales":[{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.255},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.255},{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.255}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[0,19]}},{"name":"Adults","description":"","locales":[{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.681},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.681},{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.681}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[20,49]}},{"name":"Seniors","description":"","locales":[{"name":"UnitedProvinces.Beaches","id":"UP.Beaches","population":0.063},{"name":"UnitedProvinces.Hills","id":"UP.Hills","population":0.063},{"name":"UnitedProvinces.Pastures","id":"UP.Pastures","population":0.063}],"properties":{"type":"GPWv4","gender":["m","f"],"age":[50,84]}}],"group_specifications":[],"groups_locales_parameters":[{"id":0,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.27"},{"id":1,"param":"beta","locale":"UnitedProvinces.Hills","group":"Children","value":"0.27"},{"id":2,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.27"},{"id":3,"param":"sigma","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.21739"},{"id":4,"param":"sigma","locale":"UnitedProvinces.Hills","group":"Children","value":"0.21739"},{"id":5,"param":"sigma","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.21739"},{"id":6,"param":"psym","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.63658"},{"id":7,"param":"psym","locale":"UnitedProvinces.Hills","group":"Children","value":"0.63658"},{"id":8,"param":"psym","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.63658"},{"id":9,"param":"psev","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.04865"},{"id":10,"param":"psev","locale":"UnitedProvinces.Hills","group":"Children","value":"0.04865"},{"id":11,"param":"psev","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.04865"},{"id":12,"param":"pcri","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.08557"},{"id":13,"param":"pcri","locale":"UnitedProvinces.Hills","group":"Children","value":"0.08557"},{"id":14,"param":"pcri","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.08557"},{"id":15,"param":"pdeath","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.75472"},{"id":16,"param":"pdeath","locale":"UnitedProvinces.Hills","group":"Children","value":"0.75472"},{"id":17,"param":"pdeath","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.75472"},{"id":18,"param":"ra","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.05"},{"id":19,"param":"ra","locale":"UnitedProvinces.Hills","group":"Children","value":"0.05"},{"id":20,"param":"ra","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.05"},{"id":21,"param":"ri","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.75188"},{"id":22,"param":"ri","locale":"UnitedProvinces.Hills","group":"Children","value":"0.75188"},{"id":23,"param":"ri","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.75188"},{"id":24,"param":"rm","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.125"},{"id":25,"param":"rm","locale":"UnitedProvinces.Hills","group":"Children","value":"0.125"},{"id":26,"param":"rm","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.125"},{"id":27,"param":"rs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.07143"},{"id":28,"param":"rs","locale":"UnitedProvinces.Hills","group":"Children","value":"0.07143"},{"id":29,"param":"rs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.07143"},{"id":30,"param":"rc","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.07143"},{"id":31,"param":"rc","locale":"UnitedProvinces.Hills","group":"Children","value":"0.07143"},{"id":32,"param":"rc","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.07143"},{"id":33,"param":"is","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.02643"},{"id":34,"param":"is","locale":"UnitedProvinces.Hills","group":"Children","value":"0.02643"},{"id":35,"param":"is","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.02643"},{"id":36,"param":"iqs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.07143"},{"id":37,"param":"iqs","locale":"UnitedProvinces.Hills","group":"Children","value":"0.07143"},{"id":38,"param":"iqs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.07143"},{"id":39,"param":"ie","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.09301"},{"id":40,"param":"ie","locale":"UnitedProvinces.Hills","group":"Children","value":"0.09301"},{"id":41,"param":"ie","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.09301"},{"id":42,"param":"ipre","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.16179"},{"id":43,"param":"ipre","locale":"UnitedProvinces.Hills","group":"Children","value":"0.16179"},{"id":44,"param":"ipre","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.16179"},{"id":45,"param":"iasym","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.16179"},{"id":46,"param":"iasym","locale":"UnitedProvinces.Hills","group":"Children","value":"0.16179"},{"id":47,"param":"iasym","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.16179"},{"id":48,"param":"imild","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.19231"},{"id":49,"param":"imild","locale":"UnitedProvinces.Hills","group":"Children","value":"0.19231"},{"id":50,"param":"imild","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.19231"},{"id":51,"param":"vs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":52,"param":"vs","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":53,"param":"vs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":54,"param":"vqs","locale":"UnitedProvinces.Beaches","group":"Children","value":"0"},{"id":55,"param":"vqs","locale":"UnitedProvinces.Hills","group":"Children","value":"0"},{"id":56,"param":"vqs","locale":"UnitedProvinces.Pastures","group":"Children","value":"0"},{"id":57,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Children","value":"0.0333"},{"id":58,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Children","value":"0.0333"},{"id":59,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Children","value":"0.0333"},{"id":60,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.27"},{"id":61,"param":"beta","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.27"},{"id":62,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.27"},{"id":63,"param":"sigma","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.21739"},{"id":64,"param":"sigma","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.21739"},{"id":65,"param":"sigma","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.21739"},{"id":66,"param":"psym","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.63658"},{"id":67,"param":"psym","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.63658"},{"id":68,"param":"psym","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.63658"},{"id":69,"param":"psev","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.04865"},{"id":70,"param":"psev","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.04865"},{"id":71,"param":"psev","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.04865"},{"id":72,"param":"pcri","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.08557"},{"id":73,"param":"pcri","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.08557"},{"id":74,"param":"pcri","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.08557"},{"id":75,"param":"pdeath","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.75472"},{"id":76,"param":"pdeath","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.75472"},{"id":77,"param":"pdeath","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.75472"},{"id":78,"param":"ra","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.05"},{"id":79,"param":"ra","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.05"},{"id":80,"param":"ra","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.05"},{"id":81,"param":"ri","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.75188"},{"id":82,"param":"ri","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.75188"},{"id":83,"param":"ri","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.75188"},{"id":84,"param":"rm","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.125"},{"id":85,"param":"rm","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.125"},{"id":86,"param":"rm","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.125"},{"id":87,"param":"rs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.07143"},{"id":88,"param":"rs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.07143"},{"id":89,"param":"rs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.07143"},{"id":90,"param":"rc","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.07143"},{"id":91,"param":"rc","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.07143"},{"id":92,"param":"rc","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.07143"},{"id":93,"param":"is","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.02643"},{"id":94,"param":"is","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.02643"},{"id":95,"param":"is","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.02643"},{"id":96,"param":"iqs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.07143"},{"id":97,"param":"iqs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.07143"},{"id":98,"param":"iqs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.07143"},{"id":99,"param":"ie","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.09301"},{"id":100,"param":"ie","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.09301"},{"id":101,"param":"ie","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.09301"},{"id":102,"param":"ipre","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.16179"},{"id":103,"param":"ipre","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.16179"},{"id":104,"param":"ipre","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.16179"},{"id":105,"param":"iasym","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.16179"},{"id":106,"param":"iasym","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.16179"},{"id":107,"param":"iasym","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.16179"},{"id":108,"param":"imild","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.19231"},{"id":109,"param":"imild","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.19231"},{"id":110,"param":"imild","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.19231"},{"id":111,"param":"vs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":112,"param":"vs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":113,"param":"vs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":114,"param":"vqs","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0"},{"id":115,"param":"vqs","locale":"UnitedProvinces.Hills","group":"Adults","value":"0"},{"id":116,"param":"vqs","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0"},{"id":117,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Adults","value":"0.0333"},{"id":118,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Adults","value":"0.0333"},{"id":119,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Adults","value":"0.0333"},{"id":120,"param":"beta","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.27"},{"id":121,"param":"beta","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.27"},{"id":122,"param":"beta","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.27"},{"id":123,"param":"sigma","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.21739"},{"id":124,"param":"sigma","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.21739"},{"id":125,"param":"sigma","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.21739"},{"id":126,"param":"psym","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.63658"},{"id":127,"param":"psym","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.63658"},{"id":128,"param":"psym","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.63658"},{"id":129,"param":"psev","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.04865"},{"id":130,"param":"psev","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.04865"},{"id":131,"param":"psev","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.04865"},{"id":132,"param":"pcri","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.08557"},{"id":133,"param":"pcri","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.08557"},{"id":134,"param":"pcri","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.08557"},{"id":135,"param":"pdeath","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.75472"},{"id":136,"param":"pdeath","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.75472"},{"id":137,"param":"pdeath","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.75472"},{"id":138,"param":"ra","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.05"},{"id":139,"param":"ra","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.05"},{"id":140,"param":"ra","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.05"},{"id":141,"param":"ri","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.75188"},{"id":142,"param":"ri","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.75188"},{"id":143,"param":"ri","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.75188"},{"id":144,"param":"rm","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.125"},{"id":145,"param":"rm","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.125"},{"id":146,"param":"rm","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.125"},{"id":147,"param":"rs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.07143"},{"id":148,"param":"rs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.07143"},{"id":149,"param":"rs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.07143"},{"id":150,"param":"rc","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.07143"},{"id":151,"param":"rc","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.07143"},{"id":152,"param":"rc","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.07143"},{"id":153,"param":"is","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.02643"},{"id":154,"param":"is","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.02643"},{"id":155,"param":"is","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.02643"},{"id":156,"param":"iqs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.07143"},{"id":157,"param":"iqs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.07143"},{"id":158,"param":"iqs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.07143"},{"id":159,"param":"ie","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.09301"},{"id":160,"param":"ie","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.09301"},{"id":161,"param":"ie","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.09301"},{"id":162,"param":"ipre","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.16179"},{"id":163,"param":"ipre","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.16179"},{"id":164,"param":"ipre","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.16179"},{"id":165,"param":"iasym","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.16179"},{"id":166,"param":"iasym","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.16179"},{"id":167,"param":"iasym","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.16179"},{"id":168,"param":"imild","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.19231"},{"id":169,"param":"imild","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.19231"},{"id":170,"param":"imild","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.19231"},{"id":171,"param":"vs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":172,"param":"vs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":173,"param":"vs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":174,"param":"vqs","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0"},{"id":175,"param":"vqs","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0"},{"id":176,"param":"vqs","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0"},{"id":177,"param":"gamma","locale":"UnitedProvinces.Beaches","group":"Seniors","value":"0.0333"},{"id":178,"param":"gamma","locale":"UnitedProvinces.Hills","group":"Seniors","value":"0.0333"},{"id":179,"param":"gamma","locale":"UnitedProvinces.Pastures","group":"Seniors","value":"0.0333"}],"facilities":[{"id":1,"name":"Household","description":"The household facility represents the pairwise connections between household members. Unlike schools and workplaces, everyone must be assigned to a household."},{"id":2,"name":"School","description":"The school facility represents all of the pairwise connections between people in schools, including both students and teachers. The current methods in SynthPops treat student and worker status as mutually exclusive."},{"id":3,"name":"Workplace","description":"The workplace facility represents all of the pairwise connections between people in workplaces, except for teachers working in schools. After some workers are assigned to the school contact layer as teachers, all remaining workers are assigned to workplaces. Workplaces are special in that there is little/no age structure so workers of all ages may be present in every workplace."},{"id":4,"name":"Community","description":"The community facility reflects the nature of contacts in shared public spaces like parks and recreational spaces, shopping centres, community centres, and public transportation. All links between individuals are considered undirected to reflect the ability of either individual in the pair to infect each other."}],"facilities_interactions":[{"locales":"UnitedProvinces.*","facilities":[[[0.37,0.45,0.18],[0.34,0.51,0.15],[0.26,0.29,0.45]],[[0.92,0.08,0.01],[0.65,0.35,0],[0.78,0.22,0]],[[0,0.64,0.36],[0.03,0.75,0.22],[0.04,0.69,0.27]],[[0.54,0.33,0.13],[0.15,0.61,0.24],[0.11,0.43,0.46]]]}],"facilities_timespent":[{"locales":"UnitedProvinces.*","matrix":[[0.4,0.4,0.46],[0.31,0.09,0.03],[0.08,0.31,0.27],[0.2,0.2,0.23]]}],"interaction_specifications":[],"schedules":[{"id":"5","name":"Vaccination","notes":"describe schedule for Vaccination...","detail":[{"id":1,"start_date":"2021-03-01","end_date":"2021-06-30","control_params":[{"name":"degree","value":"0"},{"name":"max_capacity","value":"5000"},{"name":"price_per_dose","value":"40"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"3","name":"Masks","notes":"describe schedule for Masks...","detail":[{"id":1,"start_date":"2021-03-01","end_date":"2021-06-30","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null},{"id":2,"start_date":"2021-07-01","end_date":"2021-12-31","control_params":[{"name":"compliance","value":"0"},{"name":"cost_per_day","value":"0.05"},{"name":"max_transmission_reduction","value":"0.5"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"6","name":"School closure","notes":"describe schedule for School closure...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"percentage","value":"0.2"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"7","name":"Workplace closure","notes":"describe schedule for Workplace closure...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"percentage","value":"0.2"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"8","name":"Mass_Screening_and_Contact_Tracing","notes":"describe schedule for Mass_Screening_and_Contact_Tracing...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"degree","value":"0.4"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]},{"id":"9","name":"Border_Closures","notes":"describe schedule for Border_Closures...","detail":[{"id":1,"start_date":"2021-01-01","end_date":"2021-12-31","control_params":[{"name":"degree","value":"0.3"},{"name":"cost_per_day","value":"1.8"}],"locales":"UnitedProvinces.*","has_trigger":false,"condition":"def isActive():\n\t# code here, this function must return a boolean value!\n\treturn True\n\t","repeat_config":{"repeat_type":"none","end_type":"never","end_date":"2021-12-31","end_times":0},"repetition_of":null}]}],"references":[],"mobility":[],"mobility_specifications":[],"border":{"data":[{"src_locale":"UnitedProvinces.Beaches","dst_locale":"UnitedProvinces.Hills","src_locale_id":"UP.Beaches","dst_locale_id":"UP.Hills","group":"*","value":0.7},{"src_locale":"UnitedProvinces.Beaches","dst_locale":"UnitedProvinces.Pastures","src_locale_id":"UP.Beaches","dst_locale_id":"UP.Pastures","group":"*","value":0.3},{"src_locale":"UnitedProvinces.Hills","dst_locale":"UnitedProvinces.Beaches","src_locale_id":"UP.Hills","dst_locale_id":"UP.Beaches","group":"*","value":0.6},{"src_locale":"UnitedProvinces.Pastures","dst_locale":"UnitedProvinces.Beaches","src_locale_id":"UP.Pastures","dst_locale_id":"UP.Beaches","group":"*","value":0.4}],"specifications":[{"id":1,"src_locale":"UnitedProvinces.*","dst_locale":"UnitedProvinces.*","group":"*","value":0,"impedance":70,"mobility_source":"GPWv4"}]},"airport":{"data":[],"specifications":[]},"facility":{"data":[],"specifications":[]},"optimize":{"configs":{"max_cost":1000000000000},"interventions":[{"name":"Vaccination","control_params":[{"name":"degree","min_value":0,"max_value":1},{"name":"max_capacity","min_value":9200,"max_value":9200},{"name":"price_per_dose","min_value":40,"max_value":40}]},{"name":"Masks","control_params":[{"name":"compliance","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":0.05,"max_value":0.05},{"name":"max_transmission_reduction","min_value":0.5,"max_value":0.5}]},{"name":"School closure","control_params":[{"name":"percentage","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]},{"name":"Workplace closure","control_params":[{"name":"percentage","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]},{"name":"Mass_Screening_and_Contact_Tracing","control_params":[{"name":"degree","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]},{"name":"Border_Closures","control_params":[{"name":"degree","min_value":0,"max_value":1},{"name":"cost_per_day","min_value":1.8,"max_value":1.8}]}]}}'''


In [11]:
PERIOD = 7 

# Env
import gym, json
from gym import spaces
from epipolicy.core.epidemic import construct_epidemic
from epipolicy.obj.act import construct_act

class EpiEnv(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['human']}

    def __init__(self, session):
        super(EpiEnv, self).__init__()
        self.epi = construct_epidemic(session)
        total_population = np.sum(self.epi.static.default_state.obs.current_comp)
        obs_count = self.epi.static.compartment_count * self.epi.static.locale_count * self.epi.static.group_count
        action_count = 0
        action_param_count =  0
        for itv in self.epi.static.interventions:
            if not itv.is_cost:
                action_count += 1
                action_param_count += len(itv.cp_list)
        self.act_domain = np.zeros((action_param_count, 2), dtype=np.float64)
        index = 0
        for itv in self.epi.static.interventions:
            if not itv.is_cost:
                for cp in itv.cp_list:
                    self.act_domain[index, 0] = cp.min_value
                    self.act_domain[index, 1] = cp.max_value
                    index += 1
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Box(low=0, high=1, shape=(action_count,), dtype=np.float64)
        # Example for using image as input:
        self.observation_space = spaces.Box(low=0, high=total_population, shape=(obs_count,), dtype=np.float64)

    def step(self, action):
        expanded_action = np.zeros(len(self.act_domain), dtype=np.float64)
        index = 0
        for i in range(len(self.act_domain)):
            if self.act_domain[i, 0] == self.act_domain[i, 1]:
                expanded_action[i] = self.act_domain[i, 0]
            else:
                expanded_action[i] = action[index]
                index += 1

        epi_action = []
        index = 0
        for itv_id, itv in enumerate(self.epi.static.interventions):
            if not itv.is_cost:
                epi_action.append(construct_act(itv_id, expanded_action[index:index+len(itv.cp_list)]))
                index += len(itv.cp_list)

        total_r = 0
        for i in range(PERIOD):
            state, r, done = self.epi.step(epi_action)
            total_r += r
            if done:
                break
        return state.obs.current_comp.flatten(), total_r, done, dict()

    def reset(self):
        state = self.epi.reset()
        return state.obs.current_comp.flatten()  # reward, done, info can't be included
    def render(self, mode='human'):
        pass
    def close(self):
        pass

jsons = [SIR_A, SIR_B, SIRV_A, SIRV_B, COVID_A, COVID_B, COVID_C]
epi_ids = ["SIR_A", "SIR_B", "SIRV_A", "SIRV_B", "COVID_A", "COVID_B", "COVID_C"]

In [12]:
index = 2
session = json.loads(jsons[index])
env = EpiEnv(session)